In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None
from IPython.core.display import HTML

HTML(open("../documents/custom.html", "r").read())

<br/>
<span style="background:#f0f0e0;padding:1em">Copyright (c) 2020-2021 ETH Zurich, Scientific IT Services. This work is licensed under <a href="https://creativecommons.org/licenses/by-nc/4.0/">CC BY-NC 4.0</a></span><br/>
<br/>

<p style="font-size: 2.5em; font-weight: bold;">Section 3: Concepts</p>

In previous Section you've learned how to identify what is making your Python program run slow.

Now it's turn to learn about common approaches to make your Python program run faster.

We will start with an overview of various code optimization strategies, in order of application preference. Afterwards, we will dive into some of few basic code optimization concepts, leaving the more conceptually and technically advanced ones for the next Sections.

# Overview: Strategies for optimization

## Avoid unnecessary computations

Not running computations saves runtime.

<table>
    <tr><td><img src="imgs/captain_obvious.gif" width="350px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://indepest.com/2021/03/14/captain-obvious/">https://indepest.com/2021/03/14/captain-obvious/</a></sub></center></td></tr>
</table>



<div class="alert alert-block alert-success">

<p><i class="fa fa-info-circle"></i>&nbsp You can avoid running unnecessary computations, for instance, by:</p>
<ol>
    <li><strong>input data pre-processing</strong> e.g. to read or to transform data, or to pre-compute useful values, using an extra data structures if needed,
        <ul>
        <li>falls into an art of using better algorithms and data structures</li>
        </ul>
    </li>
    <li><strong>pre-allocating memory</strong>, to avoid copying in-memory data e.g. in a dynamic array growing in each loop iteration,</li>
    <li><strong>saving (caching/&quot;memoizing&quot;) results of repeating computations</strong>, to avoid running multiple long-running function calls with the same input arguments.</li>
</ol>

</div>

<br/>

### Caching and memoization

**Cache** is a hardware or software component that stores data so that future requests for that data can be served faster. The data stored in a cache might be the result of an earlier computation or a copy of data stored elsewhere. 

**Memoization** is an optimization technique that stores the results of resource-consuming function calls within a lookup table, allowing subsequent calls to reuse the stored results and avoid repeated computation. Memoization is a special case of caching.

The basic memoization application to a function looks as follows:

0. Set up a cache data structure for function results
1. Every time the function is called with input arguments do one of the following:
   * A) return the cached result, if any is available for given input arguments; OR
   * B) compute the missing result, and update the cache before returning the result.

    <table>
        <tr><td><img src="imgs/memoization.png" width="800px"></td></tr>
    </table>

What can you use as a cache?

* Memory:
    * variables
    * arrays
    * dictionaries
* Data stores:
   * disk files
   * database
   * cloud storage
   
You will find already implemented caches; in Python, to memoize function results use:

* `functools.lru_cache` for memory-based caching, or
* `joblib.Memory` for file-based caching.

<br/>

## Use better algorithms and data structures

Algorithmic improvements can have the biggest benefit in improving the runtime due to reducing time (or space) complexity;

- Buying a faster expensive CPU/GPU may speedup your computations ca. $5$ to $10$ times;
- Running in parallel on a high performance computer may speedup your computations ca. $50$ to $100$ times, using a very expensive hardware;

**BUT**

- Using a better algorithm and reducing number of operations, for instance, from $2 n^3$ to $10 n^2$ for input data of size $n = 10.000$, speeds up your code $\frac{2\cdot 10^{12}}{10^{9}} = 2000$ times, without any additional hardware costs.

<table>
    <tr><td><img src="imgs/algorithms_and_data_structures.png" width="800px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://dev.to/snj/how-to-learn-data-structures-and-algorithms-an-ultimate-guide-for-beginners-2h9c">https://dev.to/snj/how-to-learn-data-structures-and-algorithms-an-ultimate-guide-for-beginners-2h9c</a></sub></center></td></tr>
</table>


<div class="alert alert-block alert-success">

<p><i class="fa fa-info-circle"></i>&nbsp For the time-consuming parts of your scientific code you can:</p>

<ol>
    <li><strong>Pick an existing better algorithm</strong> - requires some knowledge in computer science (in particular, lingo for abstract problem formulation).</li>
    <li><strong>Develop a new better algorithm</strong> - more challenging as it requires experience/practice specifically in algorithms development (many online training sites available).</li>
</ol>

<p><strong>Complexity of an algorithm is closely related to the data structures used by it</strong>. Data structures allow to simplify or abstract-out parts of the problem being solved, like memoization of intermediate computations or search for elements within the data.</p>

</div>

<br/>

## Reduce memory usage

Making use of available fast memory speeds up the computations (recall [the latency table for different types of memory operations](https://gist.github.com/jboner/2841832)).

**BUT**

As already briefly discussed in the first Section, a **too high memory usage can negatively impact runtime** either by:

* inefficient cache usage within the CPU, or by
* memory swapping (paging) using a disk.

<table>
    <tr><td><img src="imgs/memory_usage_vs_speed.jpg" width="500px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://www.techpowerup.com/234514/firefox-54-released-multi-process-optimized-memory-footprint">https://www.techpowerup.com/234514/firefox-54-released-multi-process-optimized-memory-footprint</a></sub></center></td></tr>
</table>

**In case of CPU-intensive computations, you should avoid memory swapping at all cost.**

<div class="alert alert-block alert-success">

<p><i class="fa fa-info-circle"></i>&nbsp Memory usage can be reduced using various techniques, among others:</p>

<ol>
    <li><strong>using memory-efficient data structures</strong>, such as sparse arrays for numerical computations,</li>
    <li><strong>using appropriate memory-efficient basic data types</strong>, such as fixed length strings, big-enough integer numbers, or lower precision floating point numbers,</li>
    <li><strong>explicitly using external memory</strong> for data that does not fit into the main memory.</li>
</ol>

</div>

<br/>

### Memory swapping problem*

We introduced the concept of swapping in the first Section and it is important to understand that swapping can slow down program execution significantly.

Swap space is a space on a disk which is a substitute of a fast main memory (RAM). Whenever an operating system runs short of main memory it swaps parts of the main memory (pages) with disk contents, i.e.:

1. swap out: move currently unused main memory parts (pages) to a swap space on a disk, and make the free main memory available to the currently running program;
1. swap in: read back from the swap space on a disk memory parts (pages) into the main memory, making it available to the currently running program.

<table>
    <tr><td><img src="imgs/swapping.jpg" width="400px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://www.cs.uic.edu/~jbell/CourseNotes/OperatingSystems/9_VirtualMemory.html">https://www.cs.uic.edu/~jbell/CourseNotes/OperatingSystems/9_VirtualMemory.html</a></sub></center></td></tr>
</table>


Swapping helps operating system to manage programs that use more memory than there actually is available (avoiding crashing applications instead).

**BUT**

Reading from a disk memory is slow. If swapping happens too often, especially in case **when one program uses more then available main memory, the program is mostly waiting for the operating system to swap the data** (is ["swapping to death"](https://en.wikipedia.org/wiki/Memory_paging#Swap_death)).

<br/>

## Use faster languages, like C/C++

Recall that statically typed programming languages--ones where one declares types of variables, function arguments, or return values--can be much faster than dynamically typed languages, such as Python, at a cost of pre-required compilation to a machine code.

Using fast languages for a project usually comes at cost of bigger program size and longer development-time.

<table>
    <tr><td><img src="imgs/the_computer_language_benchmarks_game-time_vs_size.png" width="1000px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://twitter.com/ChapelLanguage/status/921074120191655937/">https://twitter.com/ChapelLanguage/status/921074120191655937/</a></sub></center></td></tr>
    <tr><td><center><sub>Source<sup>2</sup>: <a href="https://benchmarksgame-team.pages.debian.net/benchmarksgame/">https://benchmarksgame-team.pages.debian.net/benchmarksgame/</a></sub></center></td></tr>
</table>

**BUT**

You can write your project in Python and still delegate a lot of (crucial) computations to actually run as C/C++, without a significant program size or development time overhead.

<div class="alert alert-block alert-success">

<p><i class="fa fa-info-circle"></i>&nbsp For your own Python programs you can employ C/C++ by:</p>

<ol>
    <li><strong>preferring vectorized C/C++-level routines</strong> using <a href="https://numpy.org/">NumPy</a> arrays
        (or <a href="https://pandas.pydata.org/">Pandas</a> data frames) over slow Python <code>for</code> loops over
        elements/rows/columns;<ul>
            <li>Note: use libraries that build on top of NumPy or Pandas, like <a href="https://scipy.org/">SciPy </a>
                or <a href="https://scikit-learn.org">Scikit-learn</a>.</li>
        </ul>
    </li>
    <li><strong>compiling your Python code to C/C++</strong> using tools such as <a
            href="https://numba.pydata.org/">Numba</a>, <a href="https://www.pypy.org/">PyPy</a>, or <a
            href="https://pythran.readthedocs.io">Pythran</a>;</li>
    <li><strong>&quot;<a href="https://en.wikipedia.org/wiki/Language_binding">binding</a>&quot; to Python your own
            C/C++ code</strong> using <a href="https://cython.org/">Cython</a>
        <ul>
            <li>Note: while C/C++ is a convenient default for binding fast crucial routines and provides various options
                for Python, you can call routines virtually from any other well-established fast languages, like, for
                instance Fortran via <a href="https://numpy.org/doc/stable/f2py/">F2PY</a>, Java via <a
                    href="https://jpype.readthedocs.io/en/latest/">JPype</a>, Rust via <a
                    href="https://pyo3.rs">PyO3</a> etc.</li>
        </ul>
    </li>
</ol>

</div>


The Section on code optimization will introduce in detail these approaches and tools.

<br/>

## Run code in parallel

The principle is simple: split work among multiple workers to reduce the total runtime.

<table>
    <tr><td><img src="imgs/serial_vs_parallel.png" width="500px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://www.teldat.com/blog/parallel-computing-bit-instruction-task-level-parallelism-multicore-computers/">https://www.teldat.com/blog/parallel-computing-bit-instruction-task-level-parallelism-multicore-computers/</a></sub></center></td></tr>
</table>


<div class="alert alert-block alert-success">

<p><i class="fa fa-info-circle"></i>&nbsp Parallelization in computing has many different levels; from the lowest level:</p>

<ol>
    <li>
        <p><a href="https://en.wikipedia.org/wiki/Bit-level_parallelism">bit-level parallelism</a> - processor operating
            on whole &quot;chunks&quot; of bits (32-bit, 64-bit);</p>
    </li>
    <li>
        <p><a href="https://en.wikipedia.org/wiki/Instruction-level_parallelism">instruction-level parallelism</a> -
            simultaneous execution of multiple processor instructions, which are optimized first by hardware or
            compilers for a maximizing average number of instructions run per CPU clock cycle (a 3.0GHz CPU performs 3
            million clock cycles per second);</p>
    </li>
    <li><a href="https://en.wikipedia.org/wiki/Data_parallelism"><strong>data parallelism</strong></a> - distributing
        the data across multiple threads/processors, which operate on the data in parallel, e.g.,<ul>
            <li><a
                    href="https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm#Parallel_and_distributed_algorithms">parallel
                    and distributed large matrix multiplication</a></li>
            <li>the <a href="https://pandas.pydata.org/docs/user_guide/groupby.html">Split-Apply-Combine</a> strategy, or the <a href="https://en.wikipedia.org/wiki/MapReduce">MapReduce programming model</a>
                <table>
                    <tr>
                        <td><img src="imgs/split-apply-combine.png" width="500px"></td>
                    </tr>
                    <tr>
                        <td>
                            <center><sub>Source: <a
                                        href="https://github.com/anurag-code/Split-Apply-Combine-Data-Mining-in-Python">https://github.com/anurag-code/Split-Apply-Combine-Data-Mining-in-Python</a></sub>
                            </center>
                        </td>
                    </tr>
                </table>
            </li>
        </ul>
    </li>
    <li><a href="https://en.wikipedia.org/wiki/Task_parallelism"><strong>task parallelism</strong></a> - distributing
        tasks across multiple threads/processors, which perform the tasks in parallel, e.g.<ul>
            <li>parallel force updates in <a href="https://en.wikipedia.org/wiki/Molecular_dynamics">molecular
                    dynamics</a>, or parallel finite element updates in a <a
                    href="https://en.wikipedia.org/wiki/Finite_element_method">finite element method</a> mesh</li>
        </ul>
    </li>
    <li>and <a href="https://en.wikipedia.org/wiki/Loop-level_parallelism">loop-level parallelism</a> - a special case
        combining task and data parallelism, where tasks and a corresponding data chunks are instructions and data used
        in <code>for</code> loop iterations, executed then in parallel.</li>
</ol>

</div>


In the three Sections on parallel computing on: 1) CPUs, 2) clusters, and 3) GPUs, you will learn how to use tools for the data and tasks parallelism.

### Lower-level of parallelism in Python*

The low bit- and instruction-levels of of parallelism can be employed only indirectly in Python; for instance:

* by use of 32-bit NumPy floating-point numbers on a 64-bit CPU,
* by use of CPU's vector instructions, called [single instruction, multiple data (SIMD)](https://en.wikipedia.org/wiki/SIMD) instructions, which can be employed by:
    * using [Numba which leverages LLVM's ability to optimize some `for` loops to use SIMD instructions](https://numba.pydata.org/numba-doc/latest/user/faq.html#does-numba-vectorize-array-computations-simd);
    * [Pythran compilation with a C++ xsimd library support](https://serge-sans-paille.github.io/pythran-stories/bye-bye-boostsimd-welcome-xsimd.html);
    * binding to a plain C/C++ code which directly uses low-level SIMD intrinsics, like [Intel's Streaming SIMD Extensions (SSE)](https://en.wikipedia.org/wiki/Streaming_SIMD_Extensions);

  see also: [*Pushing Python toward C speeds with SIMD*](https://laurenar.net/posts/python-simd/).


### Parallelization limits

Throwing more workers to work won't simply shorten the work time proportionally as there are always parts which cannot be done in parallel.

<table>
    <tr><td><img src="imgs/ParallelLimits.png" width="400px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://demotywatory.pl/913269/Polscy-robotnicy">https://demotywatory.pl/913269/Polscy-robotnicy</a></sub></center></td></tr>
</table>

**Amdahl's law** is a theoretical limit on achievable speedup of a task with a fixed problem size, when a known part of the task runtime must be spent in serial execution, and rest can be parallelized.

For example:
* program needs 20h to complete on 1 CPU,
* 2h (10%) portion of the program cannot be parallelized, whereas the remaining 18h (90%) can,
* ⇒ the minimum total execution time cannot be less than 2h
* ⇒ speedup can't be more than 10x

<table>
    <tr><td><img src="imgs/AmdahlsLaw.png" width="600px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://link.springer.com/referenceworkentry/10.1007%2F978-0-387-09766-4_77">https://link.springer.com/referenceworkentry/10.1007%2F978-0-387-09766-4_77</a></sub></center></td></tr>
</table>

We will get back to this and other measures of parallel scaling in more detail in the Section on parallel computing.

<br/>

## Use fast hardware

<table>
    <tr><td><img src="imgs/Supercomputer.png" width="350px"></td></tr>
    <tr><td><center><sub>Source: <a href="https://dribbble.com/shots/1984685-Supercomputer-visual-pun">https://dribbble.com/shots/1984685-Supercomputer-visual-pun</a></sub></center></td></tr>
</table>


<div class="alert alert-block alert-success">

<p><i class="fa fa-info-circle"></i>&nbsp You can make your program run faster by using high-end hardware, like:</p>

<ol>
    <li><strong>Multi-core servers</strong>,</li>
    <li><strong>High Performance Computing (HPC) clusters</strong> a.k.a. supercomputers,</li>
    <li><strong>Grapic cards</strong> a.k.a. GPUs.</li>
</ol>

<p>Using any of the above means running your code in parallel, but <strong>some hardware is suitable only for some problems</strong>
    (CPU-bound vs. I/O-bound problems, or problems fitting into &quot;single instruction, multiple data&quot; processing type).</p>

</div>

Most often you need to adjust your program to target specific hardware.

In the three Sections on parallel computing you will see how to adjust your Python programs for specific hardware, usually, without much overhead.

<br/>

# Algorithm complexity analysis: Big O notation

We've seen in the profiling example in the previous Section how one can empirically try to identify the relation between program's runtime and the input data size, e.g. to be roughly $n^2$, where $n$ is the input size. The program represents an algorithm that processes given input to solve a problem or perform a computation.

**Algorithms can be theoretically classified according to how their runtime or space (memory) requirements grow as the input/problem size grows**.

This is important because usually the scalability or practical applicability of an algorithm going from development tests input data to real-world large input data sets is limited by the order of growth of runtime or space requirements with respect to problem size (rather than by e.g. specific constants).

<blockquote>
    <strong>Example: algorithm complexity</strong>

| Problem size |  Algorithm 1: $n^2 / 2$ seconds runtime | Algorithm 2: $100 n$ seconds runtime |
| ------------ | --------------------------- | --------------------------- |
| $n = 10$     | 50 sec                      | 16 min 40 sec               |  
| $n = 100$    | ~1 hour 24 min              |  ~2 hours 47 min            |
| $n = 1000$   | ~5 days 19 hours            | ~1 day 4 hours              |
| $n = 10000$  | ~**1 year 30 weeks**        | ~1 week 5 days              |

If we anticipate practical problems of size $n > 1000$, we should invest into developing Algorithm 2, which is considered to be theoretically faster as it has linear $n$ and not quadratic $n^2$ runtime. If all practical problems are limited by size $n \leq 100$, it's completely fine to stick to Algorithm 1.

[Galactic Algorithms](https://en.wikipedia.org/wiki/Galactic_algorithm) are algorithms that outperform other algorithms, but     will never be used on any of the merely terrestrial data sets on Earth. E.g. optimal integer multiplication, where the numbers need to be vastly bigger than the number of atoms in the observable universe.

</blockquote>

In algorithms complexity analysis **a unit is an abstract number**, respectively, **a performed basic operation (runtime unit) and a stored basic object (space unit)**.

<blockquote>
    <strong>Example: distance matrix</strong>

Compute a matrix of squared distances between $n$ input points $x_1,x_2,\ldots,x_n$:


<math>\begin{align}\begin{bmatrix}
0 & d_{12}^2 & \dots & d_{1n}^2 \\
d_{21}^2 & 0 & \dots & d_{2n}^2 \\
\vdots&\vdots & \ddots&\vdots&  \\
d_{n1}^2 & d_{n2}^2 & \dots & 0 \\
\end{bmatrix},\end{align} </math>

where $d_{ij} = \text{dist}(x_i, x_j) = \text{dist}(x_j, x_i) = d_{ji}$, e.g. a [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between two points.

Assuming:
    
1. as a runtime operation unit a single distance computation, and
2. as a storage object unit a number representing a single distance,

one needs to compute $\frac{n\cdot (n - 1)}{2} \approx n^2\,/\,2$ distances, which are to be stored in a $n\times n = n^2$ memory array.
    
We say that both **runtime complexity** and required **space complexity** of such distance matrix computation is quadratic, or, more formally, "oh of $n$-squared", which is denoted as:
$$O(n^2)$$

.
</blockquote>

$O(\cdots)$ is a so called **Big O notation**, or which is a mathematical notation that describes the limiting behavior of a function, up to a constant, when the argument $n$ tends towards infinity (see e.g. [Big O notation @ Wikipedia](https://en.wikipedia.org/wiki/Big_O_notation) for a more formal mathematical description). The idea is to give an growth rate of required runtime or space, with respect to growing input/problem size. Thus, constant multipliers or lower order functions are ignored in the Big O notation, and we have, for example:

<math>\begin{align}
\frac{1}{2}(n^2 - 2n - 1) &= O(n^2)\\
&\text{ or}\\
2n + \log{n} &= O(n).
\end{align} </math>

The incentive is to characterize functions according to their growth rates as this is the main factor for estimating a speed or memory use of an algorithm.

The letter O is used because the growth rate of a function is also referred to as the **order of the function**.

Beware: a description of a function in terms of Big O notation formally provides only an upper bound on the growth rate of the function (hence the capital letter), e.g. we also have $n = O(n^2)$, but no one really writes that their algorithms are slower or more memory-hungry than they actually are.

## Orders of common functions

<table>
    <tr><td><img src="imgs/bigo.png" width="600px"></td></tr>
        <tr><td><center><sub>Source: <a href="https://runestone.academy/runestone/books/published/pythonds/index.html">https://runestone.academy/runestone/books/published/pythonds/index.html</a></sub></center></td></tr>
</table>

In a table form, from lowest to highest "order of growth", with additional color coding for practical applicability, and with examples of problems with a corresponding complexity:

<table >
    <thead>
        <tr style="border-bottom: 1px solid black;" >
            <th style="width:100px">Notation</th>
            <th style="width:100px">Name</th>
            <th style="width:600px">Example</th>
        </tr>
    </thead>
    <tbody>
        <tr style="background: lightgreen; ">
            <td>$O(1)$</td>
            <td><a href="https://en.wikipedia.org/wiki/Time_complexity#Constant_time">Constant</a></td>
            <td>Primitive operations<br/>Determining if a binary number is even or odd</td>
        </tr>
        <tr style="background: lightgreen; border-bottom: 1px solid black; ">
            <td>$O(\log{}n)$</td>
            <td><a href="https://en.wikipedia.org/wiki/Time_complexity#Logarithmic_time">Logarithmic</a></td>
            <td>Binary search</td>
        </tr>
        <tr style="background: yellow; ">
            <td>$O(n)$</td>
            <td><a href="https://en.wikipedia.org/wiki/Time_complexity#Linear_time">Linear</a></td>
            <td>Single loop<br/> Find element in an unsorted list<br/> Scalar/dot product (multiplication of a vector and vector)<br/> Special sorting (Counting sort)</td>
        </tr>
        <tr style="background: yellow; border-bottom: 1px solid black;">
            <td>$O(n\log{}n)$</td>
            <td><a href="https://en.wikipedia.org/wiki/Time_complexity#Linearithmic_time">Log Linear</a></td>
            <td>Sorting (Quicksort, Heapsort and Merge sort)</td>
        </tr>
        <tr style="background: orange; ">
            <td>$O(n^2)$</td>
            <td><a href="https://en.wikipedia.org/wiki/Time_complexity#Table_of_common_time_complexities">Quadratic</a>
            </td>
            <td><strong>Typical case where algorithms work well on test data or smaller data sets and start to fail for larger examples!</strong> <br/>One nested loop<br/> Find duplicate elements in a list (naive)<br/> Multiplication of a matrix and a vector<br/> Simple sorting algorithms (Bubble sort, Selection sort, Insertion sort)</td>
        </tr>
        <tr style="background: orange;  ">
            <td>$O(n^3)$</td>
            <td><a href="https://en.wikipedia.org/wiki/Time_complexity#Table_of_common_time_complexities">Cubic</a></td>
            <td>Two nested loops<br/> Matrix multiplication (naive)<br/> Solving a system of $n$ linear equations (via Gaussian elimination)</td>
        </tr>
        <tr style="background: orange; border-bottom: 1px solid black;">
            <td>$O(n^c)$, $c>1$</td>
            <td><a href="https://en.wikipedia.org/wiki/Time_complexity#Polynomial_time">Polynomial</a></td>
            <td>Finding maximum flow in a network (minimum cut in a graph)<br/> Workers to tasks Assignment Problem (maximum
                weighted bipartite matching)</td>
        </tr>
        <tr style="background: orangered;">
            <td>$O(2^n)$</td>
            <td><a href="https://en.wikipedia.org/wiki/Time_complexity#Exponential_time">Exponential</a></td>
            <td>Computing all subsets of a list<br/> Finding the (exact) solution to the Traveling Salesman Problem using dynamic programming</td>
        </tr>
        <tr style="background: orangered;">
            <td>$O(n!)$</td>
            <td><a href="https://en.wikipedia.org/wiki/Factorial">Factorial</a></td>
            <td>Computing all permutations of a list<br/> Solving the Traveling Salesman Problem via brute-force search</td>
        </tr>
    </tbody>
</table>

## Quiz exercise [15 min]

1. What is a runtime and space complexity of this function:
    ```python
    def compute(n: int):
        a = -1
        i = n
        while i > 0:
            a += 1
            i //= 2
        return a
    ```


1. What is a runtime complexity of an algorithm that performs $10^6 n$ initial operations, $2 n^3$ nested-loops operations and $10^9 \log{n}$ post-processing operations?

1. Does an $O(\log{n})$ algorithm run faster than $O(n)$ algorithm?

1. A program computes results
   - for each $n$ in $2 n$ steps,
   - but once every $n^2$ calls the program needs to run data maintenance that takes additionally $n^2 + 3$ steps. 
   
What is the runtime complexity of the program?

**Solutions**

1. Runtime complexity is $O(\log{n})$, as `i` decreases from `n`, `n/2`, `n/4`, ... to `1` ($\log_2{n}$ is "how many time to multiply by 2 to get over $n$ value").

1. $O\left(10^6 n + 2 n^3 + 10^9 \log{n}\right) = O\left(n + n^3 + \log{n}\right) = O\left(n^3\right)$

1. No, not in general, only true for large enough input size $n$, but up to some $n$ values, depending on the actual constants and lower order terms of the runtime complexity, the $O(n)$ algorithm may run faster than the $O(\log{n})$ algorithm.

1. The Big-Oh runtime complexity is the worst-case scenario, so it's $O(2 n + n^2) = O(n^2)$, but in such cases it makes sense to talk about average runtime complexity, which would be in this case 
$\frac{n^2  \cdot 2 n + n^2 + 3}{n^2} = \frac{2 n^3 + n^2 + 3}{n^2} = O(n)$

# Avoiding unnecessary computations: memoization example

<blockquote>
    <strong>Example: Fibonacci numbers</strong>

The [Fibonacci numbers](https://en.wikipedia.org/wiki/Fibonacci_number), commonly denoted $F_n$, form a sequence, called the Fibonacci sequence, such that each number is the sum of the two preceding ones, starting from $0$ and $1$. That is,

<math>\begin{align}
F_0 &= 0,\\
F_1 &= 1, \\
F_{n}&=F_{n-1}+F_{n-2},\text{ for } n > 1
\end{align}
</math>

The task is to simply compute $F_{n}$, given $n$ as an input.
    
</blockquote>

First implementation - directly from definition, a recursion:

In [ ]:
def fibonacci_recursion(n):
    if n == 0:
        return 0
    if n == 1:
        return 1

    return fibonacci_recursion(n - 1) + fibonacci_recursion(n - 2)

In [ ]:
%time fibonacci_recursion(35)

That wasn't very fast. Let's profile this implementation using `%lprun` from `line_profiler` (with some lower number to speed it up):

In [ ]:
%load_ext line_profiler 

In [ ]:
%lprun -f fibonacci_recursion fibonacci_recursion(15)

That does not look good - there are many more $F_0$ and $F_1$ checks then the actual recursion step summations $F_{n}=F_{n-1}+F_{n-2}$. Let's analyze the algorithm's complexity.

That's how the computation will look like:
```
            f(n)
          /     \
     f(n-1)     f(n-2)
    /     \    /     \
f(n-2) f(n-3) f(n-3) f(n-4)
  ...    ...    ...    ...
 /   \
f(1) f(0)
```
That's a $O\left(2^n\right)$ time and space complexity method (there are $2$ calls at each of $n$ recursion steps).

And that's a lot of repetitive calls of our function (especially `f(1)` and `f(0)` calls), which we don't really need. Let's try to avoid that!

What if we memoize the already computed results in a lookup table?

In the recursive approach, on each function call check if the result was computed for given input, and if it was, then simply read it from memory in $O(1)$ time. So the computation looks then like:
```
                  f(n)
                /     \
           f(n-1)   lookup f(n-2)
          /     \
      f(n-2) lookup f(n-3)
        ...
     f(3)
     /  \
  f(2) lookup f(1)
 /   \
f(1) f(0)
```
This would be now a $O(n)$ time (left recursion branch) and $O(n)$ space (lookup table) algorithm.

Let's try available memory- and disk-based cache implementations.

## Memoization in memory with `functools.lru_cache`

**`@functools.lru_cache`** is a built-in Python decorator that wraps a function with a cache that **saves up to the `maxsize` most recent calls in memory**. It can save time when an expensive or I/O bound function is periodically called with repetitive arguments.

An **LRU (least recently used)** cache works best when the most recent calls are the best predictors of upcoming calls (for example, the most popular articles on a news server tend to change each day). The cache’s size limit assures that the cache does not occupy too much space (for example, in long-running processes such as web servers).

Let's try the Fibbonacci's number using the LRU cache.

Second implementation - memoization:

In [ ]:
import functools


@functools.lru_cache(maxsize=128)
def fibonacci_lru(n):
    if n == 0:
        return 0
    if n == 1:
        return 1

    return fibonacci_lru(n - 1) + fibonacci_lru(n - 2)

In [ ]:
%time fibonacci_lru(35)

That was much quicker.

A `functools.lru_cache`-wrapped function has additional methods to display cache status or to clear ("invalidate") cache, respectively, `.cache_info()` and `.cache_clear()`:

In [ ]:
print(fibonacci_lru.cache_info())
# memoization-based solution used `hits` pre-computed results, and
# on each of `misses` added cache entry contributing to `currsize`

fibonacci_lru.cache_clear()
print(fibonacci_lru.cache_info())

<div class="alert alert-block alert-warning">
    <p>
        <i class="fa fa-warn"></i>&nbsp<strong>Beware</strong>:
        benchmarking memoization-based solutions is tricky - it is only really fair to <strong>measure runtime with the cache setup</strong>.
    </p>
</div>

This is what we did with timing the single call above. Next runs will be much faster as it is really only getting pre-computed value from the cache:

In [ ]:
%time fibonacci_lru(35)
%timeit -r3 fibonacci_lru(35)

`%timeit` also tries to detect caching:

In [ ]:
fibonacci_lru.cache_clear()
%timeit -r3 -n 1 fibonacci_lru(35) 

Take-home notes:

* caching using a decorator requires **only minimal code change**, and
* **in-memory caching is fast**,

BUT

The LRU or caching **limitation** in general is that **cache arguments must be hashable** (in particular, **non-mutable**), so e.g. `functools.lru_cache` is not applicable to functions with arguments with type such as `list` (but, if possible, you can use `tuple` instead).

## Memoization in files with `joblib.Memory`

[Joblib](https://joblib.readthedocs.io/) is an additional Python package that provides a set of tools for lightweight pipelining of computations. In particular it provides **disk-based caching of function calls** (memoization) via **`joblib.Memory`** class.

Fibonacci's number with disk cache:

In [ ]:
import joblib
import shutil


# for a clean demonstration: using a new temp dir as a cache dir
cachedir = "./cache_dir"

shutil.rmtree(cachedir, ignore_errors=True)

print("Caching to directory:", cachedir)

In [ ]:
memory_disk = joblib.Memory(location=cachedir, verbose=0)


@memory_disk.cache
def fibonacci_disk(n):
    if n == 0:
        return 0
    if n == 1:
        return 1

    return fibonacci_disk(n - 1) + fibonacci_disk(n - 2)

In [ ]:
%time fibonacci_disk(35)
%timeit -r3 fibonacci_lru(35)

**Disk access (I/O) is slow**, so setting up or actual reads from the `joblib.Memory` disk cache are slower than in case of a memory cache, but:

  * **cached data is available when code is run again** in a new interpreter session,
  * it is useful as a **tool for creating "checkpoints"/"snapshots"** (e.g. during workflow development),
  * `joblib.Memory` specifically works with non-hashable arguments, such as `list`.

## Coding exercise [15 min]

Avoid unnecessary computations in the Euclidian matrix example by avoiding computing `0` values and by using a memory cache to avoid computing each non-zero value twice (hint: use a built-in `sorted` function to order points). How big cache do you need?

The memoization strategy pays off only when a single distance computation takes some time. To that end, use the high dimensional points example (below).

Compare runtimes (hint: benchmark first memory cache use separately).

In [ ]:
# %load ../examples/euclidian_distance_d.py
#!/usr/bin/env python3
"""Euclidean distance example in d-dimensional space"""

import random


def setup_points(n, d=100_000):
    # create n points in d-dim for testing
    points = []
    for i in range(0, d * n, d):
        points.append(tuple(random.random() for j in range(d)))
    return points


def dist_squared(a, b):
    s = 0
    d = len(a)
    for i in range(d):
        s += (a[i] - b[i]) ** 2
    return s


def dist_matrix(points, dist_func=dist_squared):
    # compute distance matrix using given `dist_func`
    rows = []
    for p in points:
        row = []
        for q in points:
            row.append(dist_func(p, q))
        rows.append(row)
    return rows


if __name__ == "__main__":
    M = dist_matrix(setup_points(10))
    for row in M[:5]:
        print(row[:5])


In [ ]:
# Benchmarking setup
n_points = 10
points = setup_points(n_points)

In [ ]:
# Modify and benchmark distance functions

# %timeit -r 3 -n 1 dist_matrix(points, dist_func=...)

In [ ]:
# SOLUTION

# The point is not that this the optimal way to do this, but that using `if` and
# `lru_cache` is rather small and easy code modification that already improves

print(f"Problem size: n = {n_points}, d = {len(points[0])}")
print()

print("# reference")
%timeit -r 3 -n 1 dist_matrix(points)
print()


print("# no zero distance")
def dist_squared_no_zero(a, b):
    if a == b:
        return 0
    return dist_squared(a, b)

%timeit -r 3 -n 1 dist_matrix(points, dist_func=dist_squared_no_zero)
print()


print("# no zero distance and cached ordered pair results")
import functools
import time

@functools.lru_cache(maxsize=(n_points - 1) * n_points // 2)
def dist_squared_cached(a, b):
    return dist_squared(a, b)

def dist_squared_no_zero_ordered_cached(a, b):
    if a == b:
        return 0    
    p, q = sorted((a, b))  # a < b
    return dist_squared_cached(p, q)

dist_squared_cached.cache_clear()

# only call once because of caching:
%timeit -r 1 -n 1 dist_matrix(points, dist_func=dist_squared_no_zero_ordered_cached)
print()

# Python built-in data structures

We will give a quick overview of important built-in Python data structures and complexity of typical operations. We've already seen that choosing the right data structure for your task can make a significant difference.

To be able to choose well, or design well a data structure or an algorithm, you need to know the basic data structures first.

## Numbers

In [ ]:
# integer
2 + 2

In [ ]:
# float
2 + 2.0

Adding numbers and similar operations are $O(1)$, with one exception of very large integer numbers.

Contrary to compiled languages such as `C`, Python integers are implemented internally as 32 or 64 bit numbers or as arrays of these to **avoid overflow**.

In [ ]:
# fine in C and in Python:
print(2 ** 62)

# overflows in C, but not in Python:
print(2 ** 63)

In case the 64 bit implementation is not suitable, computations such as addition or multiplication do not run directly on the CPU anymore but are implemented in software. This has two consequences:

- operations become suddenly slower
- complexity is not $O(1)$ any more but depends on the number of digits of the involved numbers.

Luckily these differences are small and **non-constant operations complexity applies only to very large integer numbers** (with 19 and more digits).

**float** numbers in Python and **all dtypes in numpy** overflow and, thus, **have constant operations complexity**.

Note: if you want non-overflowing numbers in Python use the `decimal` module from the standard library. Here all operations are implemented in software and thus are always slower than working with native `float` values.

In [ ]:
import decimal

print(decimal.Decimal(2.0) ** 1100)

try:
    print(2.0 ** 1100)
except OverflowError as e:
    print("OverflowError", e)

## Strings

In [ ]:
a = "This is a string!"

print(
    len(a)
)

i = a.find("string")
print(i)

print(
    a[i:]
)

print(
    a.split()
)

print(
    a.lower().startswith("this")
)

**String is not a basic data type**. Since there is no single character type in Python string seems and feels like an atomic data type, but **string is an [array](https://en.wikipedia.org/wiki/Array_data_structure) of single characters**; as such, **operations on a string depend on its length**.

<table>
    <tr><td><img src="imgs/String.png" width="600px"></td></tr>
        <tr><td><center><sub>Source: <a href="https://en.wikipedia.org/wiki/String_(computer_science)">https://en.wikipedia.org/wiki/String_(computer_science)</a></sub></center></td></tr>
</table>

Arrays are most often stored as a one/contiguous memory segment of known size and element type, such that position of each element in the memory can be quickly computed from its index.

### Concatenation

Concatenation of two strings `a` and `b` of length $n$ and $m$, , respectively, requires to: 

1. Allocate new string `c` to hold $n + m$ bytes. 
2. Copy $n$ bytes from `a` to `c`.
3. Copy $m$ bytes from `b` to `c` (after the content from `a`).

Step 1 is usually $O(1)$, step 2 is $O(n)$ and step 3 is $O(m)$. In total this is $O(n + m)$.

**To concatenate a _hard-coded_ number of strings** use either `+`, or, even better, use [string formatting with f-strings or the `str.format` method](https://docs.python.org/3/tutorial/inputoutput.html#fancier-output-formatting), e.g. `f"{a}_{b}"` for concatenation of strings `a` and `b` with `_` as a separator.

Naively concatenating $n$ strings of length $m$ one-by-one using `+` results in a $O(m + 2m + 3m + \ldots + n\cdot m) = O(m\cdot (1+...+n)) = O(m\cdot(n+1)\cdot n/2) = O(n^2\cdot m)$ runtime complexity:

In [ ]:
import time

b = "1" * 2_000

for n in 1_000, 2_000, 4_000:
    c = ""
    t = time.time()
    for _ in range(n):
        c = c + b + ""  # `+ ""` is an anti-optimization trick; more below
    needed = time.time() - t
    print(f"concatenating {n:5d} strings of len {len(b)} took {needed:.2f} seconds")

Luckily Python has some **internal optimizations to get around this behavior** (mainly by reusing memory and avoiding / reducing copy operations) in many (**but not all**) situations

In the example above we've used `+ ""` to trick Python into not using its internal optimization.

Let's check how Python internal optimization works out-of-the box in this example:

In [ ]:
b = "1" * 2_000

for n in 1_000, 2_000, 4_000:
    c = ""
    t = time.time()
    for _ in range(n):
        c = c + b
    needed = time.time() - t
    print(f"concatenating {n:5d}  strings of len {len(b)} took {needed:.4f} seconds")

In more complicated situations it is  **not easy to predict when internal optimizations are done**.

Instead, **to concatenate a _variable_ number of strings use `join`**:
1. collect the strings in a list `l` of strings (if you don't have this list already)
2. use `"".join(l)` to concatenate strings.

Comment: `s.join(l)` concatenates the strings from `l` with the separator string `s` between them.

Preparing list for `join` takes $O(n)$ on average, and using `join` allows to pre-allocate final memory and do copy operations once for each of $n$ input string, giving a runtime complexity $O(n\cdot m)$

In [ ]:
b = "1" * 2_000

for n in 1_000, 2_000, 4_000:
    l = []
    s = time.time()
    for _ in range(n):
        l.append(b)
    c = "".join(l)
    needed = time.time() - s
    print(f"concatenating {n:5d}  strings of len {len(b)} took {needed:.4f} seconds")

### Operations

Let us assume that `a` and `b` are Python strings of length $n$ and $m$, respectively; then:

| Operation | Comment | Complexity | |
| - | - | - | - |
| `a = "..."` | create string | $O(1)$ | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `len(a)`    | length of string  | $O(1)$ |  |
| `a[i]` | read string character | $O(1)$ |  |
| `a + b`| concatenate two strings| $O(n+m)$ |  |
| `"".join([b_1, ..., b_n]) ` | concatenate $n$ strings | $O(n\cdot m)$ |  |
| `a.find(b)`/`a.index(b)` | find index of substring | $O(n\cdot m)$ | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>2</sup></i> <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>3</sup></i> |
| `b in a`    | check if substring is in a string | $O(n\cdot m)$ | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>2</sup></i> <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>3</sup></i> |
| `a.split(b)` | split string using substring as separator | $O(n+m)$ |  |
| `a.lower()/a.upper()` | transform string characters to lower/upper case | $O(n)$ |  |
| `a.startswith(b)/a.endswith(b)` | check if string starts/ends with substring | $O($$\min{(n,m)}$$)$|  |

<i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> Creation of hard-coded stings (literals) `a = "..."` is $O(n)$ when the Python bytecode (`.pyc` file) is compiled first by a Python interpreter, but afterwards, the pointer access is $O(1)$.

<i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>2</sup></i> `b in a` or `a.find(b)`/`a.index(b)`, are implemented such that they often run faster than $O(n\cdot m)$ ([read here](https://web.archive.org/web/20100221040018/http://effbot.org/zone/stringlib.htm)).

<i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>3</sup></i> For a pattern-based string search use the regular expression methods from the [`re` standard library module](https://docs.python.org/3/library/re.html). They are heavily optimized but there is no theoretical run-time guarantee in $O$ notation.

The non-didactic "string as basic type" design, combined with many built-in string utilities, is, arguably, one of the cornerstones of Python popularity. Be sure to check out other [string methods of Python's standard library](https://docs.python.org/3/library/stdtypes.html#string-methods).

## Lists

In [ ]:
x = list(2*i for i in range(5))
# equivalently:
#   x = [0, 2, 4, 6, 8]
# equivalently:
#   x = [2*i for i in range(5)]
print(x)

x.append(x[-1] + 2)
print(x)

del x[0]
print(x)

print(
    4 in x
)
print(
    x.index(4)
)

Python lists are the general workhorse in Python for "collecting data".

Python lists are implemented using [dynamic arrays](https://en.wikipedia.org/wiki/Dynamic_array) (and not using [linked lists](https://en.wikipedia.org/wiki/Linked_list)). Dynamic arrays have spare space for new elements and are copied into 1.5-2 times larger array when the spare space runs out.

<table>
    <tr><td><img src="imgs/Dynamic_array.svg" width="250px"></td></tr>
        <tr><td><center><sub>Source: <a href="https://en.wikipedia.org/wiki/Dynamic_array">https://en.wikipedia.org/wiki/Dynamic_array</a></sub></center></td></tr>
</table>

The benefit of using dynamic arrays are low runtime complexities for access to list elements and, on average, for adding list elements, but this comes at the cost of some memory overhead (in Python around 13%).

### Operations

Let us assume that `x` is a Python list of length $n$, `i` an arbitrary integer, `y` an arbitrary Python object and `it` an arbitrary iterable. Then:

| Operation | Comment | Complexity | |
| - | - | - | - |
| `x = list(it)` | create list | $O(\text{len}(\text{it}))$ |  |
| `len(x)`    | length of list  | $O(1)$ |  |
| `x.append(y)`| append element| $O(1)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `x.extend(it)` | extend list | $O(\text{len}(\text{it}))$ on average |  |
| `x[i] = y` | overwrite list element | $O(1)$ |   |
| `y = x[i]` | read list element | $O(1)$ |  |
| `del x[i]` | remove element | $O(n)$ ($O(1)$ for the last element)| |
| `y = x.pop(i)` | read element and remove | $O(n)$ ($O(1)$ for the last element)| |
| `x.insert(i, y)` | insert element | $O(n - i)$ | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>2</sup></i> |
| `x.index(y)` | find index of element | $O(n)$ |  |
| `y in x`    | check membership | $O(n)$ |  |

<i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> Appending and element to a list takes constant time  $O(1)$ in most situations. On rare occasions the internal data needs to be reorganized which takes $O(n)$ time. The average runtime is $O(1)$ (cf. [amortized cost analysis for dynamic array](https://en.wikipedia.org/wiki/Amortized_analysis#Dynamic_array)).

<i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>2</sup></i>
Inserting an element at the beginning of the list using `x.insert(0, y)` is $O(n)$. If reading, appending or deleting both ends of a list are common operations in your problem use a `deque` (*double ended queue*) data structure from the [collections module](https://docs.python.org/3/library/collections.html#collections.deque); they all have runtime $O(1)$. The caveat is that operations on `deque` inner elements like `x[i] = y` or `y = x[i]` are $O(n)$.

### Tuples

Tuples are immutable lists, so there are no element insertion or deletion operations, but otherwise behave just like lists.

In [ ]:
x = tuple(2*i for i in range(5))
# equivalently:
#   x = (0, 2, 4, 6, 8)
print(x)

print(
    4 in x
)
print(
    x.index(4)
)

Tuples can be slightly faster than lists, but replacing tuples by lists for reasons of performance is most often an unnecessary micro-optimization.

Tuples are "hashable" so can be used e.g. as dictionary keys.

## Dictionaries

In [ ]:
x = dict((2*i, i) for i in range(5))
# equivalently:
#   x = {0: 0, 2: 1, 4: 2, 6: 3, 8: 4}
# equivalently:
#   x = {2*i: i for i in range(5)}
print(x)

x[2*5] = 5
print(x)

del x[0]
print(x)

print(
    4 in x
)
print(
    5 in x.values()
)
print(
    x.get(5, None)
)

Dictionaries implement lookup tables and are heavily optimized in Python.

- dictionary **values can be arbitrary** Python objects,
- dictionary **keys must be immutable** Python objects (such as `int`, `str`, `tuple`s of immutable objects, **not**: `list`s, `set`s, `dict`s).
- dictionary **keys are unique**.


Dictionaries are also used inside the Python interpreter in many places.

E.g.

In [ ]:
import binascii

binascii.__dict__

So when you e.g. use `binascii.hexlify`, the Python interpreter actually accesses `binascii.__dict__["hexlify"]`.

Thus having a fast and efficient dictionary implementation is crucial for the overall speed of Python.

### Operations

Let us assume that `x` is a Python dictionary with $n$ entries, `k` an arbitrary object which can be used as key, `y` an arbitrary Python object and `it` an arbitrary iterable.

| Operation | Comment | Complexity | |
| - | - | - | - |
| `x = dict(it)` | create dictionary | $O(\text{len}(\text{it}))$ | |
| `len(x)`   | size of dictionary | $O(1)$ | |
| `x[k] = y` | insert/overwrite value at key | $O(1)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> | 
| `y = x[k]`/`y = x.get(k)` | lookup value at key | $O(1)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `del x[k]` | remove value and key | $O(1)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `k in x`   | key membership test | $O(1)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `x.keys()` | keys of dictionary | $O(1)$ | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>2</sup></i> |
| `x.values()` | values of dictionary | $O(1)$ | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>2</sup></i> |
| `x.items()` | key and value pairs of dictionary | $O(1)$ | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>2</sup></i> |

<i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> The internal data structure used for dictionaries is a so-called [Hash table / Hash map](https://en.wikipedia.org/wiki/Hash_table) (for Python-specific). This data structure makes lookup, insertion and deletion operations $O(1)$ on average (using so called hashing of keys and open addressing strategy to resolve hash conflicts; [read more](http://thepythoncorner.com/dev/hash-tables-understanding-dictionaries/)).

<i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>2</sup></i> A `for` loop over `.keys()`, `.values()` or `.items()` is still $O(n)$.


### `dict.setdefault` and `Counter`


Python's standard library offers a class which adds some convenience on-top of dictionaries to make your live easier: the `collections.Counter` class.



[`Counter`](https://pymotw.com/3/collections/counter.html) takes a collection of data / an iterable and computes a dictionary which maps each item from the data to the number of its occurrences:

In [ ]:
from collections import Counter

items = list(range(5)) + list(range(4)) + list(range(2))
print(items)
print()

counter = Counter(items)
print(counter)

**Note**: We also introduced
 [`defaultdict`](https://pymotw.com/3/collections/defaultdict.html) during previous versions of the script but `defaultdict` can lead to hard to debug issues when not handled carefully, e.g. when passing a `defaultdict` to a function which expects a proper `dict`. Instead the `dict.setdefault` method can be used:
 
This methods sets a default value for unknown keys and returns the updated dictionary:

In [ ]:
d = {3 : 9}
print(d.setdefault(2, 4))  # creates new entry and creates new value 4
print(d.setdefault(3, 10))  # key exists already and thus returns 9
print(d)

## Sets

In [ ]:
a = set(i//2 for i in range(5))
# equivalently:
#   a = set([0, 0, 1, 1, 2])
print(a)

print(
    1 in a
)

a.add(3)
print(a)

a.remove(0)
print(a)

b = a.union(set([3, 4])).difference(set([0, 1]))
print(b)
print(a.symmetric_difference(b))

Python sets are a fast lookup-based collections. Recall, that we've already used sets in Section 2 to optimize our profiling example.

A set in Python represents a mathematical set. Contrary to lists / tuples:

1. There are **no duplicate elements** in a set.
2. Set has **no order**; e.g., you can not ask for the first element of set.
3. Set **elements must be immutable** (same as for dictionary keys).

Sets can be cleverly used e.g.

In [ ]:
def has_only_unique_elements(li):
    return len(set(li)) == len(li)


def count_duplicate_elements(li):
    return len(li) - len(set(li))


print(has_only_unique_elements([1, 2, 3]))
print(has_only_unique_elements([1, 2, 3, 1]))

print(count_duplicate_elements([1, 2, 3, 1, 2, 1, 4]))

### Operations

Let us assume that `a` and `b` are Python sets of size $n$ and $m$, respectively; then:

| Operation | Comment | <div style="width:200px">Complexity</div> | |
| - | - | - | - |
| `a = set(it)` | create set | $O(\text{len}(\text{it}))$ | |
| `len(a)`   | size of set | $O(1)$ | |
| `a.add(x)` | add element | $O(1)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `x in a` | element membership test | $O(1)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `a.remove(x)` | remove element | $O(1)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `a \| b`, `a.union(b)`  | set union  | $O(n + m)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `a - b`, `a.difference(b)`  | set difference  | $O(n)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `a ^ b`, `a.symmetric_difference(b)` | set symmetric difference | $O(n + m)$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |
| `a & b`, `a.intersection(b)`  | set intersection  | $O(\min(n, m))$ on average | <i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> |

<i class="fa fa-exclamation-triangle" aria-hidden="true"><sup>1</sup></i> As with dictionaries, sets are implemented in Python using hash tables (with dummy values and some optimizations for that), making lookup (membership test), insertion and deletion operations $O(1)$ on average, as well as all other size-dependent operations on average, as they use the lookup.


### Frozen sets

The built-in `frozenset` is to `set` as `tuple` is to `list` - an immutable variant of the corresponding collection type, that does not allow for element addition or removal but, e.g., can be used as a dictionary keys or other sets elements.

In [ ]:
a = frozenset(i//2 for i in range(5))
# equivalently:
#   a = frozenset([0, 0, 1, 1, 2])
print(a)

print(
    1 in a
)

# Since the union etc. operations create new sets, the result is also a frozenset
b = a.union(set([3, 4])).difference(set([0, 1]))
print(b)

## Coding exercise [15 min]

Given two strings `s1` and `s2` of words which are separated by spaces, find words which are unique in each string and only appear in one of the strings; for instance:
```python
s1 = "you used to code in MATLAB but then you switched to Python"
s2 = "we used to program in MATLAB but then we switched to Python"

unique_words_in_strings(s1, s2)
```
should return any of:
```python
['code', 'program']
['program', 'code']
```

Hint: use lookup-based structures such as `set` or `Counter`.

What is the time and space complexity of your solution?

In [ ]:
from collections import Counter


def _unique_words(words, excluded_words):
    
    out = []
    
    unique_words = ...
    ...
    for word in unique_words:
        if ...:
            out.append(word)
    
    return out


def unique_words_in_strings(s1, s2):

    s1_words = ...
    s2_words = ...

    return _unique_words(s1_words, s2_words) + _unique_words(s2_words, s1_words)


# s1 = "you used to code in MATLAB but then you switched to Python"
# s2 = "we used to program in MATLAB but then we switched to Python"

# unique_words(s1, s2)

In [ ]:
# SOLUTION

from collections import Counter


def _unique_words(words, excluded_words):
    
    unique_words = set(words) - set(excluded_words)  # O(n + m) time and mem
    words_counter = Counter(words)  # O(n) time
    
    out = []
    # O(n + m) space and time:
    for word in unique_words:
        if words_counter[word] == 1:
            out.append(word)
    return out

def unique_words_in_strings(s1, s2):

    # worst-case: one letter words only (string size equals half of the words)
    s1_words = s1.split()  # O(n) time and mem
    s2_words = s2.split()  # O(m) time and mem

    # O(n + m) time and mem (output)
    return _unique_words(s1_words, s2_words) + _unique_words(s2_words, s1_words)
    # Total complexity: O(n+m) time and mem


# Bonus: using once set symmetric difference and a single list comprehension,
#        instead of twice set difference, list comprehension, and additional list
#        concatenation
#        => smaller hidden complexity constant
def unique_words_in_strings_2(s1, s2):
    
    # worst-case: one letter words only (string size equals half of the words)
    s1_words = s1.split()  # O(n) time and mem
    s2_words = s2.split()  # O(m) time and mem

    # worst-case: each word occurs exactly once
    unique_words = set(s1_words) ^ set(s2_words)  # O(n + m) time and mem
    
    s1_counter = Counter(s1_words)  # O(n) time and mem
    s2_counter = Counter(s2_words)  # O(m) time and mem
    
    return [
        w for w in unique_words
        if w in s1_counter and s1_counter[w] == 1 or s2_counter[w] == 1
    ] # O(n+m) time and mem

    # Total complexity: O(n+m) time and mem

s1 = "you used to code in MATLAB but then you switched to Python"
s2 = "we used to program in MATLAB but then we switched to Python"

print(unique_words_in_strings(s1, s2))
print(unique_words_in_strings_2(s1, s2))

# the constant imporovement gets more visible with bigger input
%timeit unique_words_in_strings(s1*100, s2)
%timeit unique_words_in_strings_2(s1*100, s2)

## Optional coding exercise

Can you now implement an optimized version of `count_common` from Script 2 such that duplicates in both collections are also counted correctly?

In [ ]:
# Solution
from collections import Counter

def count_common(data_1, data_2):
        
    # o(n)
    c = Counter(data_2)
    counts = 0
    
    # o(n)
    for item in data_1:
        counts += c[item]
    return counts
        
    
print(count_common([1, 2, 1, 2, 3], [3, 4, 1, 1, 7]))

# Using better algorithms and data structures: examples

Recall, that in the Fibonacci's number example, we improved the time complexity of a brute-force recursive method from $O\left(2^n\right)$ to $O(n)$ by a memoization-based solution with $O(n)$ space complexity.

But we actually can do even a bit better memoizing only the last two Fibonacci's numbers, and, thus, replacing the recursion with a straightforward `for` loop:
```
iteration 1:   f(2) = f(1) + f(0)
iteration 2:   f(3) = f(2) + f(1)
...
iteration n-1: f(n) = f(n-1) + f(n-2)
```

Third implementation - sum up in a loop:

In [ ]:
def fibonacci_loop(n):
    if n == 0:
        return 0
    if n == 1:
        return 1

    prev_prev_fib = 0
    prev_fib = 1
    for i in range(2, n + 1):
        fib = prev_fib + prev_prev_fib
        prev_prev_fib = prev_fib
        prev_fib = fib
    return fib

In [ ]:
%time fibonacci_loop(35)

It is now a $O(n)$ time (`for` loop) and $O(1)$ space (three variables) algorithm.

The loop solution not only uses less memory than the memoization-based solution, but is actually also a bit faster in practice (as no time is needed for setting up the cache data structure).

Let's get some ideas for what algorithmic improvements may entail on example of two basic computing problems: sorting and searching.

## Sorting

### Selection sort

A simple method to sort a list of numbers is the [selection sort algorithm](https://en.wikipedia.org/wiki/Selection_sort):

<table>
    <tbody>
    <tr>
        <td style="font-size:120%; vertical-align:top; horizontal-align:left; width: 450px">
            <ol>
                <li>
                    <p>find the smallest number starting at position 0 and swap this with the entry at position 0.</p>
                </li>
                <li>
                    <p>find the smallest number starting at position 1 and swap this with the entry at position 1.</p>
                </li>
                <li>
                    <p>find the smallest number starting at position 2 and swap this with the entry at position 2.</p>
                </li>
                <li>
                    <p>...</p>
                </li>
            </ol>
        </td>
        <td style="width: 300px;">
            <center>
                <img src="imgs/selection_sort.jpg" />
                <sub>Source: <a href="https://stackoverflow.com/questions/36700830/selection-sort-algorithm">https://stackoverflow.com/questions/36700830/selection-sort-algorithm</a></sub>
            </center>
        </td>
    </tr>
    </tbody>
</table>


In [ ]:
import random


def selection_sort(data):
    # as described above
    for starting_position in range(len(data) - 1):
        index = find_position(data, starting_position)
        swap(data, index, starting_position)


def find_position(data, starting_position):
    # we start with assuming that the value at best_idx is the smallest
    # value
    best_idx = starting_position
    smallest_value = data[starting_position]

    # ... and the we iterate over the rest of the list updating best_idx:
    for idx in range(starting_position + 1, len(data)):
        if data[idx] < smallest_value:
            smallest_value = data[idx]
            best_idx = idx
    return best_idx


def swap(data, index_1, index_2):
    data[index_1], data[index_2] = data[index_2], data[index_1]


random.seed(42)  # for reproducibility

data = list(range(20))
random.shuffle(data)
print(data)

selection_sort(data)
print(data)

<p style="font-size: 120%"><strong>Runtime analysis</strong><sup>*</sup></p>


The runtime complexity analysis for this algorithm applied to a list of length $n$ is as follows:

We count the operations in each step which depend on $n$ and name the remaining number of operations per step as $n_0$ (e.g. array access, increasing loop counter, swap).

- The first step needs $n - 1$ comparisons and $n_0$ other operations
- The second step needs $n - 2$ comparisons and $n_0$ other operations
- The third step needs $n - 3$ comparisons and $n_0$ other operations
- ...
-  The $n-1$st step needs $1$ comparisons and $n_0$ other operations.

This is in total

$$
   (n - 1 + n_0)  + (n - 2 + n_0) + \ldots + (1 + n_0)
$$

The number of terms in parenthesis is $n-1$ and separating $n_0$ plus rearranging terms leads to

$$
(n - 1) + (n - 2) + \ldots + 1 + \,\,\, (n-1) n_0 
$$

Using some math for summing up the first $n - 1$ natural numbers this is the same as

$$
\frac{n (n - 1)}{2} + (n-1) n_0 = \frac{n^2}{2} + \frac{n - 1}{2} \left(1 + 2 n_0 \right)
$$


We can ignore the second summand $\frac{n - 1}{2} \left(1 + 2 n_0 \right)$ which is linear in $n$ and grows slower than $n^2$,

we further assume that each  operation (depending if it is comparison, swap, ...) has a runtime between $t_0$ and $t_1$ and thus the total runtime $T(n)$ is bounded by

$$
t_0 \left\{ \frac{n^2}{2} + \ldots \right\} \le T(n) \le t_1 \left\{ \frac{n^2}{2} + \ldots \right\}
$$

Since we omit constants and lower order terms in the $O$ notation we conclude that 
$$T(n) = O(n^2)$$ for the runtime of selection sort.

### Can we do better?

Yes we can! Practically important sorting algorithms have a runtime complexity $O(n \log  n)$ which grows **much slower** than $O(n^2)$. 

**Note**: $\log n$ is proportional to the number of digits of $n$ and thus $O(n \log n)$ is often said to be "almost linear".

Known standard algorithms from this class are

- [Merge sort](https://en.wikipedia.org/wiki/Merge_sort) is a [stable](https://en.wikipedia.org/wiki/Sorting_algorithm#Stability) sorting algorithm, with $O(n \log n)$ runtime and $O(n)$ memory requirements.</br>
  It can sort efficiently data which does fit into memory. Conceptually, you divide data into smaller sub-problems until you reach single elements, which are sorted, and then re-assemble sorted lists by zipping them together simultaneously.

<table>
<tr><td><img src="imgs/merge_sort.png" width="600px"></td></tr>
<tr><td><center><sub>Source: <a href="https://en.wikipedia.org/wiki/Merge_sort">https://en.wikipedia.org/wiki/Merge_sort</a></sub></center></td></tr>
</table>

- [Quick sort](https://en.wikipedia.org/wiki/Quicksort) is not stable, with $O(n \log n)$ runtime **on average** and $O(\log n)$ memory requirements.</br>
  In corner cases such as presorted data or reversed presorted data the runtime can degrade to $O(n^2)$. Nevertheless quick sort is used often in practice due to its lower memory requirement and lower (hidden) constants in the runtime complexity compared to merge sort. Efficient implementations of quicksort are among the fastest sorting algorithms in practice. 
- [Timsort](https://en.wikipedia.org/wiki/Timsort)  is stable, with $O(n \log n)$ worst-case runtime and $O(n)$ worst-case memory requirement.</br>
  It is a hybrid algorithm, derived from merge sort and insertion sort ($O(n^2)$ algorithm), designed to perform well in practice.
  It has small hidden complexity constants and is fast on naturally occurring (partially) pre-sorted data.</br>
  **This is the sorting algorithm used in Python** (since version 2.3 and became meanwhile also the sorting algorithm used in Java and some other programming languages).




In [ ]:
import random
import time

k = 500_000

random.seed(42)  # for reproducibility

for n in 1_000, 2_000, 4_000:
    l = [random.randint(0, k) for i in range(n)]
    
    t = time.time()
    selection_sort(l)
    needed = time.time() - t
    print(f"selection sort of list of len {n} took {needed:.2f} seconds")

print()

for n in 100_000, 200_000, 400_000:
    l = [random.randint(0, k) for i in range(n)]
    
    t = time.time()
    l = sorted(l)
    needed = time.time() - t
    print(f"timsort of list of len {n} took {needed:.2f} seconds")


### Special sorting algorithms

One can prove that a **sorting algorithm which is based on comparing items can not be faster than $O(n \log n)$**.

**But**: can you sort items without comparing them?

Yes, under some extra assumptions, you actually can sort items without comparing them.

<p style="font-size: 120%">Example: <a href="https://en.wikipedia.org/wiki/Counting_sort"><strong>Counting sort</strong></a></p>

Counting sort algorithm **assumes that the items to sort belong to a limited set $S$ of $N$ items with known order**; e.g. the numbers $0 \ldots N -1$. If the maximum item is not given, you can always first find it in $O(n)$ time.

We continue to show case how this sort works using numbers.

The idea is as follows:

1. Initialise array of counts for every item in $S$ with `0` values.
1. Run once over the input numbers increasing count for every seen item.
2. Iterate over the $S$ counts array, from smallest to largest number, and append each number to the result as many times as it was seen in the input.

E.g the list `[3, 2, 1, 0, 1, 3, 5, 1, 2, 3]` would result in the following counts and output lists:

<table>
    <tr><td><img src="imgs/algorithms-counting_sort.png" width="500px"></td></tr>
</table>

In [ ]:
# assume domain S is integer numbers 

def counting_sort(numbers):
    # find N = max(S)
    # O(n) time, O(1) space
    N = max(numbers) + 1

    # count items
    # O(n) time, O(N) space
    counts = [0] * N
    for number in numbers:
        counts[number] += 1

    # output items
    # O(N) time, O(n) space
    result = []
    for i in range(N):
        result.extend([i] * counts[i])

    return result

    # total: O(n + N) time, O(n + N) memory


numbers = [3, 2, 1, 0, 1, 3, 5, 1, 2, 3]

counting_sort(numbers)

The runtime complexity of this algorithm is $O(n + N)$ with a "hard" $O(n + N)$ memory requirement. This is only practical with a relatively small $N$;

Take-home notes:

* We **leverage additional knowledge** of the sort domain to get the $O(n)$ performance, at a cost of a potentially much bigger space requirement.
* For problems with an already "fast" algorithms available, quite often there is a **trade-off between space and runtime**, i.e. reducing runtime requires additional space, and vice-versa.

Other notable sorting algorithms which do not rely on comparisons are [bucket sort](https://en.wikipedia.org/wiki/Bucket_sort) and [radix sort](https://en.wikipedia.org/wiki/Radix_sort).

All these algorithms do not work on general data and can include a significant memory overhead!

### How to sort in Python?

The previous explanations served the purpose to introduce runtime analysis and some sorting basics. In practice you should use available sorting algorithms:

1. `sorted()` or `list.sort()` for sorting arbitrary data in Python; see [Python Documentation HOWTO on sorting](https://docs.python.org/3/howto/sorting.html#sortinghowto).
2. `numpy.sort()` for numerical data, which also offers $O(n)$ radix sort for integers (see [`numpy.sort` documentation](https://numpy.org/doc/stable/reference/generated/numpy.sort.html)).

Implementing your own sorting algorithm can be fun and insightful but you can not expect that your result will outperform these implementations!

## Searching

Searching algorithms can be divided into exact and approximate matching.

### Exact searching

Python offers several methods to find an element in a given data set.

**List lookup**

In [ ]:
words = ["ab", "def", "ghi", "xyz"]

print("ghi" in words)
print(words.index("ghi"))

Both operations are $O(n)$ if $n$ is the size of the data set.

**Dictionary lookup**

In case the items in our collection are unique (no duplicates) we can search faster.

Either we reduce lookup time by using a dictionary (assuming elements are immutable):

In [ ]:
words_indices = {}
for position, word in enumerate(words):
    words_indices[word] = position

print("ghi" in words_indices.keys())
print(words_indices["ghi"])

Here the preprocessing time is $O(n)$ (with $O(n)$ extra memory), but all subsequent lookups are $O(1)$.

**Set lookup**

In case you only want to check for membership, you can also use a set:

In [ ]:
words_unique = set(words)
print("ghi" in words_unique)

This gives also $O(1)$ lookup, but has lower memory requirements than a dictionary (by a constant factor).

**Binary search**

In case your data set is sorted you can use binary search from the `bisect` module from the standard library:

In [ ]:
import bisect


words = sorted(words)


def get_index_of(element, sorted_sequence):
    i = bisect.bisect_left(sorted_sequence, element)
    # all sequence elements in positions:
    #     < i are smaller
    #    >= i are greater or equal
    if i >= len(sorted_sequence) or sorted_sequence[i] != element:
        raise ValueError(f'"{element}" not found')
    return i


print(get_index_of("ghi", words))

The binary search algorithm implements the way we would naively search in a phone book or encyclopedia:

1. look in the middle,
2. if the entry at this position is the entry we are looking for we are done;
3. otherwise, if the entry at this position is "smaller" than the entry we are looking for, we restrict search to the 2nd half and repeat from 1. with a "reduced" collection.
4. else, we restrict search to the 1st half and repeat from 1. with this "reduced" collection.

<table>
    <tr><td><img src="imgs/algorithms-binary_search.png" width="800px"></td></tr>
</table>

Notes:
* if the item was not found you end up with a position where the number should have been, if it would be present in the input;
* in implementation we do not actually "reduce the data set" - we only narrow down lower or upper index to indicate section of the data that is to be used in a recursive call.

For run-time analysis we can establish the relation $T(n) = T(n / 2) +  c$ because we can reduce the data set "virtually" by a factor of 2 in every step, having some fixed cost $c$ in every iteration. Using the so called [master theorem](https://en.wikipedia.org/wiki/Master_theorem_(analysis_of_algorithms)) we can conclude that this algorithm has run-time complexity $O(\log n)$.

**Summary**

Assuming list of $n$ element as input, this is an exact search summary table sorted by increasing memory requirements (which you can neglect in most situations):

| method | runtime &nbsp; &nbsp; | works with duplicates |  requires | returns position |
|--------|---------|-----------------------|-----------------|------------------|
| list lookup | $O(n)$ | yes |  - | yes |
| bisection | $O(\log n)$ | yes |  data sorting $O(n \log n)$ | yes |
| set | $O(1)$ | no | set construction $O(n)$ | no |
| dict| $O(1)$ | no | dict construction $O(n)$ | yes |

### Approximate search*

Approximate search is used to find elements in a collection which are **close** to a given element. In most cases closeness is measured by an mathematical distance, such as $|a - b$| for numbers or $\|a - b \|_p$ ($p = 2$ is euclidean distance) for multi-dimensional data points.

#### Searching numbers

**List lookup**

Using a list we return numbers within the given proximity tolerance threshold:

In [ ]:
numbers = [(i % 5) / 10 for i in range(10)]
print(numbers)

In [ ]:
def find_approx(numbers, number, tolerance):
    """find numbers which deviate from `number` by
    max distance `tolerance`"""
    positions, matches = [], []
    for position, current_number in enumerate(numbers):
        if abs(number - current_number) <= tolerance:
            positions.append(position)
            matches.append(current_number)

    return positions, matches


positions, matches = find_approx(numbers, 0.25, tolerance=0.1)
print("found", matches, "at positions", positions)

Runtime of this approach is $O(n)$.

**Binary search**

Using binary search we:

1. require the input numbers to be sorted,
2. have to adjust the search for the tolerance threshold - we can look for index where minimum `number - tolerance` number would go and return numbers until we get over maximum `number + tolerance` (or vice versa)

In [ ]:
sorted_numbers = sorted(numbers)
print(sorted_numbers)
print()

In [ ]:
def find_approx_sorted(sorted_numbers, number, tolerance):
    # results will be in the interval
    # [number - tolerance, number + tolerance]

    # find first element >= number - tolerance:
    current_index = bisect.bisect_left(sorted_numbers, number - tolerance)
    positions, matches = [], []

    # go through subsequent sorted elements while tolerance is kept
    while (
        current_index < len(sorted_numbers)  # check if not end first!
        and abs(sorted_numbers[current_index] - number) <= tolerance
    ):
        positions.append(current_index)
        matches.append(sorted_numbers[current_index])
        current_index += 1

    return positions, matches


positions, matches = find_approx_sorted(sorted_numbers, 0.25, tolerance=0.1)
print("found", matches, "at positions", positions)

Runtime of this approach is $O(\log n + N)$ for sorted data of size $n$ and $N$ matches.

In case your data is not sorted from the beginning the complexity of a single search is $O(n \log n)$ which looks worse than the simple list approach from before. 

But if you do **many look-ups** $k$ in the same data it can be much faster as data is sorted once; assuming tolerance low enough to make the number of matches in each lookup $N$ negligible, the complexity in multi-lookup case is $O(n \log n + k \log n)$, which is linear in $k$.

#### Searching n-dimensional data points

Approximate search in $n$-dimensional euclidean spaces is a common task in some fields such as machine learning (e.g. the [kNN classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)), [geographic information systems (GIS)](https://en.wikipedia.org/wiki/Geographic_information_system), [computer graphics](https://en.wikipedia.org/wiki/Space_partitioning#In_computer_graphics) and [computer games](https://en.wikipedia.org/wiki/Collision_detection#Spatial_partitioning).

The standard algorithms for n-dimensional point search boil down to building and using special data structures, analogously to dictionary-based exact search of an element's position.

The main data structures from this domain are so called [k-d trees](https://en.wikipedia.org/wiki/K-d_tree).


<table>
    <tr><td><img  src="imgs/3dtree.png" width="300" ></td></tr>
        <tr><td><center><sub>Source: <a href="https://en.wikipedia.org/wiki/K-d_tree">https://en.wikipedia.org/wiki/K-d_tree</a></sub></center></td></tr>
</table>




For $n$ data points, which are randomly distributed $k$-d trees offer nearest point lookup in $O(\log n)$ time and an (axis-parallel) range lookup returning $m$ points in $O\left(n^{1−\frac{1}{k}} + m\right)$ time. Performance **degrades in higher dimensions** and for some particular data distributions. General runtime analysis is difficult.

Such data structures also require time to build, e.g. from a Python list or from numpy arrays. Depending on the implementation setting up a $k$-d tree takes $O(n \log n)$ or $O(n \log^2 n)$ time: [see also here](https://en.wikipedia.org/wiki/K-d_tree#Complexity).

Python implementations of $k$-d trees can be found
- in `scipy`: [scipy.spatial.cKDTrees](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.cKDTree.html#scipy-spatial-ckdtree), this implementation also supports multiple core computations for many lookups and matching two data sets for finding pairs of close points.
- in `scikit-learn`: [sklearn.neighbors.KDTree](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree)

In [ ]:
import numpy as np
from scipy.spatial import cKDTree

# matrix of 100 points in 3d space
np.random.seed(42)  # for reproducibility
points = np.random.random(size=(100, 3))

tree = cKDTree(points)

# find points around center which max distance 0.25
center = np.array([0.5, 0.5, 0.5])
rows = tree.query_ball_point(center, r=0.25)

print("matching points")
print(points[rows])

print("actual distances")
print(np.sum((points[rows] - center) ** 2, axis=1) ** 0.5)

#### Approximate point search for a-priori known distance tolerance*

In case one runs many queries to match points in $n$-dimensional space ($n$ small) and one uses a constant search tolerance, one can use the following technique. This can be handy to match data points from 2 collections, as e.g. when matching $(rt, m)$ peaks from an LCMS device or star catalogs in astronomy.

We demonstrate the idea for two dimensional point collections:

In [ ]:
import itertools



def preprocess(points, tolerance):
    """we 'round' points to a n-d grid with grid length
    tolerance"""
    
    lookup = {}
    for point in points:
        # must also be hashable:
        grid_point = tuple((point / tolerance).astype(np.int64))
        lookup.setdefault(grid_point, []).append(point)
    return lookup


def search(point, lookup, tolerance):
    """find all points in lookup which are close to `point` up
    to given tolerance`
    """
    
    result = []
    
    center_grid_point = (point / tolerance).astype(np.int64)
    
    # we have to iterate over neighbours also, might get
    # false positives though:
    for grid_point in _neighbours(center_grid_point):
        candidates = lookup.get(grid_point, [])
        
        # filter out false positives:
        for candidate in candidates:
            if np.linalg.norm(point - candidate) < tolerance:
                result.append(candidate)
    return result
    
def _neighbours(grid_point):
    """iterate over all neihbouring cells of the point"""
    # O(3^dim(point) !!!!! So this becomes infeasible in high dimensions:
    for offset in itertools.product(*((-1, 0, 1),) * len(grid_point)):
        yield tuple(grid_point + offset)
            
        
np.random.seed(42)
points_1 = np.round(np.random.random((10, 3)), 3)
points_2 = points_1 + 0.01 * np.ones((10, 3))

tolerance = 0.5
lookup = preprocess(points_1, tolerance)
for point in points_2:
    matches = search(point, lookup, tolerance)
    print("point", point, "has", len(matches), "matches")

**Comment**: in case the points are scattered over a space which is much larger  than the search tolerance, this technique is efficient since the `candidate` list will be small. This performance degrades to $O(n)$ in the worst-case situation that all points are rounded to the same grid point.

**Comment**: Compared to the k-d trees, the setup of `lookup` is $O(n)$, thus very fast, but only works in case the search tolerance is `fixed`.

## Coding exercise [15 min]

Given a list of $n$ words and another input word, find the number of anagrams of the input word there are in the list of words, ignoring case of the letters.

Anagram is a word formed by rearranging the letters of a different word, e.g., `"Santa"` is an anagram of `"Satan"`.

Importantly, the list of words is static (always the same), so you can do any kind of preprocessing of it, to speed up the queries.

For instance:
```python
words = [
 "AA",
 "Santa",
 "Satan",
 "a",
 "an",
 "ant",
 "antas",
 "ants",
 "as",
 "at",
 "sat",
 "ta",
 "tan",
 "tans",
]

query_words = preprocess_words(words)

count_anagrams("aa", query_words)
count_anagrams("Ant", query_words)
count_anagrams("rant", query_words)
count_anagrams("santa", query_words)
```
should return, respectively:
```
1
2
0
3
```

What is the time and space complexity of your solution when doing $k$ queries?

In [ ]:
# SOLUTION

from collections import Counter


def _fingerprint(word):
    """
    returns fingerprint which is the same for anagrams and differs else
    return value is also immutable and can be used as a key in a 
    dictionary. O(m log m) time and O(m) space for m = len(word).
    """   
    return "".join(sorted(word.lower()))
    
def preprocess_words(words):
    # O(n) time, O(n) space
    return Counter(_fingerprint(word) for word in words)


def count_anagrams(word, query_words):
    # O(1) time, O(1) space
    return query_words[_fingerprint(word)]

# k queries take, with pre-processing:
#    O(n + k) time, and O(n) space


words = [
 "AA",
 "Santa",
 "Satan",
 "a",
 "an",
 "ant",
 "antas",
 "ants",
 "as",
 "at",
 "sat",
 "ta",
 "tan",
 "tans",
]

query_words = preprocess_words(words)

print(count_anagrams("aa", query_words))
print(count_anagrams("Ant", query_words))
print(count_anagrams("rant", query_words))
print(count_anagrams("santa", query_words))

## Coding exercise [homework]

Compute intersection of two lists of integers. Each element of the intersection must appear as many times as it shows in both arrays and order of elements in the results does not matter. For instance:
```python
ints1 = [1, 2, 2, 1, 3]
ints2 = [2, 3, 2]

intersect_ints(ints1, ints2)
```
can return either of:
```python
[2, 2, 3]
[2, 3, 2]
[3, 2, 2]
```

What is the time and space complexity of your solution?

In [ ]:
# SOLUTION
# Approach 1: sorting input to reduce problem to a known one - zip-merging of two sorted lists
#             (but outputing only when equal)
def intersect_ints_sort(ints1, ints2):
    out = []
    
    # sorting: O(n log n), O(n) space (alt. O(1) space when allowed in-place)
    ints1_sorted = sorted(ints1)  # alt. in-place: `ints1.sort()`
    ints2_sorted = sorted(ints2)  # alt. in-place: `ints2.sort()`
    
    # "merge" sorted lists, appending to output only when ints are equal:
    #     O(n_max) time => worst-case: all element of longer list are smaller
    #     O(n_min) space (output) => worst-case: smaller list is a subset of longer list
    n1, n2 = len(ints1), len(ints2)
    i1, i2 = 0, 0
    while i1 < n1 and i2 < n2:
        num1, num2 = ints1_sorted[i1], ints2_sorted[i2]
        if num1 < num2:
            i1 += 1
        elif num2 < num1:
            i2 += 1
        else: # equal
            out.append(num1)
            i1 += 1
            i2 += 1

    return out

    # total: O(n_max log(n_max)) time, O(n_min) space
    #
    # Note: O(n_max) time, O(n_min) space if input lists are sorted
    #       => same as more general counter-based solution


# Approach 2: better, using a counter w/ a list size discrepancy optimisation
from collections import Counter


def intersect_ints(ints1, ints2):
    out = []
    
    # Ensure: n1 <= n2, where n1 = len(ints1), n2 = len(ints2)
    # O(1)
    ints_short, ints_long = (
        (ints1, ints2) if len(ints1) < len(ints2)
        else (ints2, ints1)
    )
    
    # Build counter
    # O(n1) time, O(n1) space
    seen = Counter(ints_short)
    
    # Check bigger list for elements, updating counter
    # O(n2) time, O(n1) space (output)
    for num in ints_long:
        if seen.get(num, 0) > 0:  # O(1)
            out.append(num)  # average O(1); we could also pre-alloc n1 mem
            seen[num] -= 1  # O(1)

    return out

    # total: O(n_max) time, O(n_min) space


ints1 = [1, 2, 2, 1, 3]
ints2 = [2, 3, 2]


print(intersect_ints_sort(ints1, ints2))
print(intersect_ints(ints1, ints2))

# Note: difference in the algorithms complexity order kicks-in only with a sufficiently large input
%timeit intersect_ints_sort(ints1*500, ints2*50)
%timeit intersect_ints(ints1*500, ints2*50)

## Other data structures

We've already mentioned some either Python-specific or more abstract data structures, like Python dictionaries (hash tables) or $k$-d tree. There are many more - cf. [Wikipedia's list of data structures](https://en.wikipedia.org/wiki/List_of_data_structures). We won't cover them here, but it's highly recommended to get familiar with at least few additional basic and common ones data structures:
* [Stacks](https://en.wikipedia.org/wiki/Stack_(abstract_data_type)) and [queues](https://en.wikipedia.org/wiki/Queue_(abstract_data_type)),
  * cf. [`collections.deque`](https://docs.python.org/3/library/collections.html#collections.deque),
* [Trees](https://en.wikipedia.org/wiki/Tree_(data_structure)),
  * ([self-balancing](https://en.wikipedia.org/wiki/Self-balancing_binary_search_tree)) [binary search tree](https://en.wikipedia.org/wiki/Binary_search_tree),
  * [B-tree](https://en.wikipedia.org/wiki/B-tree),
  * [heap](https://en.wikipedia.org/wiki/Heap_(data_structure)),
* [Graphs](https://en.wikipedia.org/wiki/Graph_(abstract_data_type)) (adjacency list, adjacency matrix)
  * cf. [NetworkX package](https://networkx.org/).


## Other algorithms

We recommend getting familiar with some general **[algorithm design paradigms](https://en.wikipedia.org/wiki/Algorithmic_paradigm)**, that help developing own algorithms; some important and common ones are:
* **[Divide-and-conquer algorithms](https://en.wikipedia.org/wiki/Divide-and-conquer_algorithm)**, where you split recursively your problem into independent smaller  problems,
    * like Merge sort;
* **[Dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming)**, where you also split your problem into same but smaller problems; the difference to divide-and-conquer is that your sub-problems overlap so you need to build-up the final solution,
    * like computing the Fibonacci's number;
* **[Backtracking](https://en.wikipedia.org/wiki/Backtracking)**, where you "track" history of your algorithm steps and return to a previous "branching" step ("backtrack") as soon as you find the there is no solution in the current branch,
    * like in solving Sudoku by picking one (e.g. lowest) of the many remaining alternative numbers, or
    * like in a [depth-first search in a graph](https://en.wikipedia.org/wiki/Depth-first_search).
* ...

The field of algorithms is vast, [wikipedia offers this overview](https://en.wikipedia.org/wiki/List_of_algorithms).

Some fields to mention are:

- [Graph algorithms](https://en.wikipedia.org/wiki/Travelling_salesman_problem). E.g. [topological sort](https://en.wikipedia.org/wiki/Topological_sorting) is used in workflow engines (e.g. [snakemake](https://snakemake.github.io)) to figure out the order of steps to execute based on their dependencies. The [Traveling salesman problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem) also requires a graph, and the algorithms named [A*](https://en.wikipedia.org/wiki/A*_search_algorithm) and [D*](https://en.wikipedia.org/wiki/D*) have been widely used, resp., in games and in mobile robots and autonomous vehicle navigation. [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) finds the shorted path between nodes in a graph, e.g. in navigation systems.
- [String search algorithms](https://en.wikipedia.org/wiki/String-searching_algorithm) to find substrings in a (long) string, e.g. the [Boyer-Moore algorithm](https://en.wikipedia.org/wiki/Boyer–Moore_string-search_algorithm)
- [Approximate string matching algorithms](https://en.wikipedia.org/wiki/Approximate_string_matching) find strings that match a pattern approximately (rather than exactly) e.g. to compensate for typos.
- [Sequence alignment](https://en.wikipedia.org/wiki/Sequence_alignment) algorithms, most known are the [Needleman-Wunsch](https://en.wikipedia.org/wiki/Needleman–Wunsch_algorithm) and [Swith-Waternam](https://en.wikipedia.org/wiki/Smith–Waterman_algorithm#Explanation) algorithms.
- [Numerical algorithms](https://en.wikipedia.org/wiki/Numerical_analysis#Areas_of_study) for numerical computations such as curve fitting, solving linear equations, ...
- [Optimization algorithms](https://en.wikipedia.org/wiki/Mathematical_optimization) (may overlap with numerical algorithms), e.g. [Gradient descent](https://en.wikipedia.org/wiki/Gradient_descent), the [Simplex algorithm](https://en.wikipedia.org/wiki/Simplex_algorithm) or the [BFGS method](https://en.wikipedia.org/wiki/Broyden–Fletcher–Goldfarb–Shanno_algorithm).
- [Combinatorical optimisation algorithms](https://en.wikipedia.org/wiki/Combinatorial_optimization) to solve the [Knapsack](https://en.wikipedia.org/wiki/Knapsack_problem) or [Traveling salesman](https://en.wikipedia.org/wiki/Travelling_salesman_problem) problems. Many of these problems require $O(2^n)$ (**WHICH IS VERY VERY BAD**) operations and can be solved pragmatically if approximate solutions are acceptable.



# Reduce memory usage

Let's dive into some common techniques of reducing memory usage in scientific computing.

It's gonna be convenient to measure memory use of a single object.

**Measuring memory consumption of an object**

In addition to `memory-profile` (FIXME: add link) introduced in Section 2 one can measure the size of a specific object.

<div class="alert alert-block alert-warning">
    <p><i class="fa fa-warning"></i>&nbsp;
       <strong>Beware</strong>:
        The standard library offers a function <code>sys.getsizeof</code>, which you should avoid, whenever possible; see: <a href="https://nedbatchelder.com/blog/202002/sysgetsizeof_is_not_what_you_want.html"><em><code>sys.getsizeof</code> is not what you want</em></a>.
    </p>
</div>

Instead, we can use the external package `pympler` to measure (approximate) size, in bytes, of an object in memory:

In [ ]:
from pympler.asizeof import asizeof

print(asizeof([]))
print(asizeof(1))
print(asizeof([1, 2]))

## Sparse arrays 

A **[sparse matrix or sparse array](https://en.wikipedia.org/wiki/Sparse_matrix)** is a matrix in which most of the elements are zero. The opposite of this is called a **dense matrix / array**.

So instead of storing all entries of a matrix it can be more efficient to **store only the positions of the nonzero entries and the corresponding nonzero values**. E.g.

`[0, 0, 0, 0, 1.25, 0, 3.4, 0]` could be represented as `[(4, 1.25), (6, 3.4)]`. Such representation is known as coordinate format (COO).

<table>
<tr><td><img src="imgs/sparse_matrix-coo.gif" width="600px"></td></tr>
<tr><td><center><sub>Source: <a href="https://matteding.github.io/2019/04/25/sparse-matrices/">https://matteding.github.io/2019/04/25/sparse-matrices/</a></sub></center></td></tr>
</table>


This technique not only reduces memory usage but can speed up mathematical operations like matrix-vector, matrix-matrix of vector-vector products significantly. E.g. adding up the entries of a sparse vector is $O(n_*)$ with $n_*$ is the number of nonzero entries, compared to $O(n)$ for a dense vector.


For some special sparse matrices also [very fast solvers for linear systems are known](https://en.wikipedia.org/wiki/Conjugate_gradient_method).

Sparse matrices play a crucial role in the [numerical solution of partial-differential equations](https://en.wikipedia.org/wiki/Finite_element_method), which are used in many simulations e.g. weather forecasts or virtual crash tests. 


### Sparse vectors and matrices in `scipy`*

`scipy` offers [different storage formats for sparse matrices](https://docs.scipy.org/doc/scipy/reference/sparse.html), e.g. the [CSR (Compressed Sparse Row)](https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format))  matrix format, which, in contrast to CCO, gives fast row access and fast matrix-vector multiplications.

<table>
<tr><td><img src="imgs/sparse_matrix-csr.gif" width="800px"></td></tr>
<tr><td><center><sub>Source: <a href="https://matteding.github.io/2019/04/25/sparse-matrices/">https://matteding.github.io/2019/04/25/sparse-matrices/</a></sub></center></td></tr>
</table>

These formats can be used for vectors and matrices, but not for multi dimensional arrays with dimensions $\gt$ 2.

[scipy.sparse.linalg](https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html#module-scipy.sparse.linalg) also offers special implementations of many common numerical algorithms optimized for sparse matrices, such as computing eigenvectors or solving linear systems.

Here we showcase how to use this:

In [ ]:
import sys

import numpy as np
import pandas as pd
from pympler.asizeof import asizeof
from scipy.sparse import csr_matrix

# create a matrix with only few random nonzero entries
np.random.seed(43)
array = np.zeros((20, 20))
for i in range(13):
    row, col = np.random.randint(0, 20, size=(2,))
    array[row, col] = 100.0 + i

print()
print("       numpy array size:", asizeof(array))

# convert matrix to a compressed sparse row (csr) matrix
scipy_sparse_array = csr_matrix(array)
print("scipy sparse array size:", asizeof(scipy_sparse_array))

**Note**: the code above constructs sparse matrix from a dense matrix, which still takes "a lot" of memory. The proper way to setup a sparse matrix from scratch is a bit more cumbersome:

In [ ]:
data, rows, cols = [], [], []

# create a compressed sparse row (csr) matrix with only few random nonzero entries
# prepare a "sparse" list of rows, columns and nonzero values
np.random.seed(43)
for i in range(13):
    row, col = np.random.randint(0, 20, size=(2,))
    rows.append(row)
    cols.append(col)
    data.append(100.0 + i)

# create the csr matrix from the list of nonzero entries
scipy_sparse_array_2 = csr_matrix(
    (np.array(data), (np.array(rows), np.array(cols))), shape=(20, 20)
)

print("scipy sparse array size:", asizeof(scipy_sparse_array_2))

# sanity check for equality
# Note: it's cheap to check inequality of sparse matrices
#       => check number of nonzero elements (nnz) in inequality sparse matrix
print("scipy sparse arrays equal:", (scipy_sparse_array != scipy_sparse_array_2).nnz == 0)

**Warning**: The random seed for construction of the entries in the previous example was carefully chosen to avoid duplicate entries in `rows`, `cols`; otherwise, duplicate entries values are added up:

In [ ]:
# place number 1, then number 2 at position (0, 0)
data = [1, 2]
rows = [0, 0]
cols = [0, 0]
print(
    csr_matrix(
        (np.array(data), (np.array(rows), np.array(cols))), shape=(2, 2)
    ).todense()
)

`scipy` offers conversion between different sparse formats and from and to dense arrays. Sparse arrays can be used in the same way as dense arrays. 

`scikit-learn` [also supports sparse matrices to represent features](https://scikit-learn.org/stable/modules/feature_extraction.html#sparsity). E.g. encoding texts using [vector space models](https://en.wikipedia.org/wiki/Vector_space_model).

### Sparse vectors in `pandas`*

**Warning**: `pandas` support [sparse vectors (1-dimensional `SparseArray`)](https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#sparsearray), but not sparse matrices.

Contrary to `scipy.sparse` arrays, the "common value", also called a "fill value", is not required to be `0`, but can be any arbitrary value.

In [ ]:
import numpy as np
import pandas as pd
from pympler.asizeof import asizeof

vector = np.zeros((1000,))
nnz = 40  # number non-zero

vector[np.random.randint(len(vector), size=nnz)] = 42
print(vector[:100])
print()
print("numpy vector size:", asizeof(vector))

pandas_sparse_vector = pd.arrays.SparseArray(vector, fill_value=0)  # 
print("pandas sparse vector size:", asizeof(pandas_sparse_vector))
print()
print(pandas_sparse_vector[:100])

Since `pandas` only supports sparse vectors, the internal format is also more space efficient compared to `scipy`:

In [ ]:
from scipy.sparse import csr_matrix

scipy_sparse_vector = csr_matrix(vector)
print("scipy sparse vector size:", asizeof(scipy_sparse_vector))

`pandas.DataFrame` can store any column as `pandas.arrays.SparseArray`, in particular all columns if you work on matrices

**BUT**

there is an overhead for storing each sparse data structure, so prefer long columns:

In [ ]:
from scipy.sparse import csc_matrix

numpy_dense_vector = pandas_sparse_vector.to_dense()

for n_col in (10, 100):

    numpy_dense_matrix = numpy_dense_vector.reshape(numpy_dense_vector.size // n_col, n_col)
    print("                matrix shape:", numpy_dense_matrix.shape)
    scipy_sparse_matrix = csc_matrix(numpy_dense_matrix)
    print("    scipy sparse matrix size:", asizeof(scipy_sparse_matrix))

    pandas_df = pd.DataFrame(numpy_dense_matrix)
    print(" pandas dense dataframe size:", asizeof(pandas_df))

    pandas_sparse_df = pd.DataFrame.sparse.from_spmatrix(scipy_sparse_matrix)
    print("pandas sparse dataframe size:", asizeof(pandas_sparse_df))
    print(pandas_sparse_df)
    print()

### Other options*

The Python package **[`sparse`](https://sparse.pydata.org/en/latest/)** supports sparse data containers which can be used like `numpy` arrays and supports **arbitrary dimensions**.

## Appropriate numerical data type in NumPy

We will learn more about `numpy` in the next script, but we anticipate that `numpy` arrays consume much less memory than Python lists.


In [ ]:
n_el, n_min, n_max = 500, -1000, 1000

import numpy as np
from pympler.asizeof import asizeof


print(f"0...{n_max} sequence container size:")

python_list = list(np.random.randint(low=n_min, high=n_max, size=n_el))
print("* list:\t\t\t", asizeof(python_list))

np_array_int = np.array(python_list, dtype=int)
print(f"* numpy {np_array_int.dtype} array:\t", asizeof(np_array_int))

NumPy supports a much greater variety of numerical types than Python does. You can reduce the amount of memory by using these types.

In our previous example the default `int` type takes 8 bytes (64 bit), whereas absolute values of our numbers are $\leq$ 1000 and thus can be represented using 11 bytes ($2^{10} = 1024$, plus a byte for a sign). The smallest suitable [`numpy` data type](https://numpy.org/doc/stable/user/basics.types.html) is `np.short`, which is an alias for `np.int16`:

In [ ]:
np_array_short = np.array(python_list, dtype=np.short)
print(f"* numpy {np_array_short.dtype} array:\t", asizeof(np_array_short))

# Sanity check: we did not "cut" any number value
print(all(np_array_int == np_array_short))

This reduced allocated memory already by a factor or ~4.

Note: when dealing with positive numbers, you can use an unsigned variant of the numeric type (e.g. `np.ushort`/`np.uint16`); unsigned types do not use less bytes, but they do increase the maximal value that can be stored (twice, as the "leading" sign byte is used as a binary number digit instead).

Use `np.iinfo` function to check min/max value of a NumPy integer type:

In [ ]:
print(np.iinfo(np.int64))
print(np.iinfo(np.int16))
print(np.iinfo(np.uint16))

The default floating-point type in `numpy` is `numpy.double` (`numpy.float64`), thus such a number consumes 64 bit or 8 bytes.

Choosing `numpy.single` (`numpy.float32`) instead leads to 50% memory reduction **at the cost of reduced accuracy**.

### A few words about floating-point accuracy

To store real numbers on a computer with finite memory (infinite memory not invented as of 2021), real numbers need to be approximated using so called [floating-point numbers](https://docs.python.org/3/tutorial/floatingpoint.html). See also [this article from Python documentation](https://docs.python.org/3/tutorial/floatingpoint.html) and [this Wikipedia article](https://en.wikipedia.org/wiki/Floating-point_arithmetic).

You can see that e.g. $\pi$ is approximated, we know that $\sin(\pi) = 0$, but:

In [ ]:
print(np.pi)
# print with 32 digits after decimal point:
print("%.32f" % np.sin(np.pi))
# print first significant digit in the scientific notation
print("%.1g" % np.sin(np.pi))

This number $0.0000000000000001...$ differs from $0$ only at 16 position after the decimal place, but it is not exactly $0$:

In [ ]:
np.sin(np.pi) == 0

<div class="alert alert-block alert-warning">
    <p>
        <i class="fa fa-warning"></i>&nbsp<strong>Beware</strong>:
        Real numbers computations on computers are <strong>approximate computations</strong>.
    </p>
</div>

Approximate computations means propagating errors, which, in turn, may lead to unexpected behavior:

In [ ]:
x = 0.0
while x != 1:
    x += 1/7
    print(x)
    if x > 1:  # too far
        break

([A real-life consequences of floating-point arithmetic errors might be quite serious](https://en.wikipedia.org/wiki/Floating-point_arithmetic#Incidents).)

Let's call $rd(x)$ the representation of a real number $x$ in a given floating-point format. We define a [**machine epsilon**](https://en.wikipedia.org/wiki/Machine_epsilon) $\epsilon$ as the **maximum relative error of a real number representation**, i.e.:
<br/><br/>
$$
\epsilon = \max_x \frac{|\mathrm{rd}(x) - x|}{|x|}
$$

The more bytes a floating-point type uses the better the representation precision.

With a **a floating-point type precision** or **trusted digits** we refer to the number of digits that we can trust to be correctly represented (starting with the first non-zero/significant digit); this is $\left\lfloor -\log_{10}{\epsilon} \right\rfloor$.

Using the `np.finfo` function to check info on a NumPy floating point type, including machine epsilon, precision, or min/max value:

In [ ]:
print(np.finfo(np.double))

The following table summarizes the most common NumPy floating-point types and their main properties:

In [ ]:
from IPython.display import display, Markdown

idouble = np.finfo(np.double)
isingle = np.finfo(np.single)
ihalf = np.finfo(np.half)

display(Markdown(f"""
| name   | `np.ndtype`         | trusted digits      | $\epsilon$        | max / -min        |
| --     | --                  | --                  | --                | --                |
| double | `{idouble.dtype!s}` | {idouble.precision} | {idouble.eps:.2e} | {idouble.max:.2e} |
| single | `{isingle.dtype!s}` | {isingle.precision} | {isingle.eps:.2e} | {isingle.max:.2e} |
| half   | `{ihalf.dtype!s}`   | {ihalf.precision}   | {ihalf.eps:.2e}   | {ihalf.max:.2e}   |
"""))

Note: in practice, you may need a bit more digits to encounter directly the representation issue, depending on the specific number values, e.g:

In [ ]:
a = 1234567890123456.1
b = 1234567890123456  # difference on 17-th digit
print(f"{a}=={b}:", a == b)
print()
print(f"np.float64({a}) == np.float64({b}):", np.float64(a) == np.float64(b))  # same as built-in `float`
print()

a = 12345678.1
b = 12345678  # difference on 9-th digit
print(f"np.float32({a}) == np.float32({b}):", np.float32(a) == np.float32(b))  
print()

a = 1234.1
b = 1234  # difference on 5-th digit
print(f"np.float16({a}) == np.float16({b}):", np.float16(a) == np.float16(b)) 
print()

You can save space by using an appropriate data type, but **be aware of the resulting limitations of numerical range and precision**.

So what to do in cases, where you consciously accept the chosen floating-point type limitations, but still want to compare for numbers equality?

<div class="alert alert-block alert-warning">
    <p>
        <i class="fa fa-warning"></i>&nbsp<strong style="font-size:120%;">Important</strong>
    </p>
    <p>
        <strong>You must NOT compare floating-point numbers exactly</strong>, i.e. using equality operator <code>==</code>.
    </p>
    <p>
        Instead, check for floating-point numbers closeness in terms of their relative or absolute difference using functions such as <a href="https://docs.python.org/dev/library/math.html#math.isclose">math.isclose</a>, <a href="https://numpy.org/doc/stable/reference/generated/numpy.isclose.html#numpy.isclose">numpy.isclose</a>, or <a href="https://numpy.org/doc/stable/reference/generated/numpy.allclose.html#numpy.allclose">numpy.allclose</a>.
    </p>
</div>

In [ ]:
print(np.sin(np.pi) == 0.0)
print(np.isclose(np.sin(np.pi), 0.0, atol=0))

In [ ]:
import numpy as np

x = 0.0
while not np.isclose(x, 1, atol=0):
    x += 1/7
    print(x)

## External memory data: example

**External memory data processing** or **out of memory processing** is done when data are too large to fit into a computer's main memory; most often, data is kept on a disk and read in blocks fitting main memory.

E.g. given a huge text file with a fixed number of numbers per line, we can determine the largest number without loading all values into memory.

Let's fabricate a dummy input file first:

In [ ]:
import numpy as np


N = 50_000  # nr of lines
M = 50  # nr of columns

with open("numbers.csv", "w") as fh:
    for i in range(N):
        np.savetxt(fh, np.random.rand(1, M) * M, delimiter = ",", fmt="%18.16f")

# file size
!du -h numbers.csv

To determine the  largest number it is sufficient to inspect line per line:


In [ ]:
maximum = None

with open("numbers.csv") as fh:
    for line in fh:
        number = max(float(n) for n in line.split(","))
        if maximum is None or number > maximum:
            maximum = number

print("largest number is", maximum)

`pandas` supports chunk-wise reading CSV or similar data files. Chunks are `pandas.DataFrame` objects and a chunk size determines numbers of lines to read at once:

In [ ]:
import pandas as pd

for chunk in pd.read_csv("numbers.csv", header=None, chunksize=5):
    print(chunk.iloc[:,:5])
    break

This can be used to determine the largest number by reading chunks of `10_000` lines:

In [ ]:
maximum = None

for chunk in pd.read_csv("numbers.csv", header=None, chunksize=10_000):
    max_in_chunk = float(chunk.max().max())
    if maximum is None or max_in_chunk > maximum:
        maximum = max_in_chunk

print("largest number is", maximum)

**Remember**: external memory data processing is suitable for data that does not fit memory; it's faster to process data in-memory.

### Reading file multiple times: example*

Sometimes you might have to run multiple times over the same file to process external memory data.

E.g. to scale a file of numbers to the range 0..1 you 
   1. First iterate over the data to determine the minimal $x_\min$ and maximal value $x_\max$.
   2. During a second iteration you replace every $x$ by $\frac{x - x_\min}{x_\max - x_\min}$  and write this number to an output file.

In [ ]:
import time
import numpy as np
import pandas as pd


start_time = time.time()

minimum = maximum = None

with open("numbers.csv") as fh:
    for line in fh:
        for n in line.split(","):
            number = float(n)
            if maximum is None or number > maximum:
                maximum = number
            if minimum is None or number < minimum:
                minimum = number

print("minimum", minimum)
print("maximum", maximum)

with open("numbers.csv") as fh_in:
    with open("numbers_scaled.csv", "w") as fh_out:
        for line in fh_in:
            line_array = np.array([float(n) for n in line.split(",")]).reshape(1, M)
            scaled = (line_array - minimum) / (maximum - minimum)
            np.savetxt(fh_out, scaled, delimiter = ",", fmt="%18.16f")

elapsed_time = time.time() - start_time
print(f"rescaling done in {elapsed_time} sec")

print(next(pd.read_csv("numbers_scaled.csv", header=None, chunksize=5)).iloc[:,:5])
!du numbers_scaled.csv
!rm numbers_scaled.csv

The `pandas` chunk-based variant uses a `.to_csv()` method for writing, with a keyword arguments:
* `mode` (Python write mode), which when set to `"a"` appends results to a given file name (or a file handle);
* `index` which when set to `None` prevents printing row names (which by default are row numbers).

In [ ]:
import time
import pandas as pd

n_chunks = 5  # try different nr of chunks
chunksize = N / n_chunks

start_time = time.time()

minimum = maximum = None

for chunk in pd.read_csv("numbers.csv", header=None, chunksize=chunksize):
    max_in_chunk = float(chunk.max().max())
    min_in_chunk = float(chunk.min().min())
    if maximum is None or max_in_chunk > maximum:
        maximum = max_in_chunk
    if minimum is None or min_in_chunk < minimum:
        minimum = min_in_chunk

print("minimum", minimum)
print("maximum", maximum)

for chunk in pd.read_csv("numbers.csv", header=None, chunksize=chunksize):
    scaled_chunk = (chunk - minimum) / (maximum - minimum)
    scaled_chunk.to_csv("numbers_scaled_pandas.csv", header=None, index=None, mode="a", float_format="%18.16f")

    # Note: you could keep the file handle open for all writes,
    #       but time to get a file handle is negligible as compared to I/O time
    # with open("numbers_scaled_pandas.csv", "a") as fh_out:
    #     chunk.to_csv(fh_out, ...)

elapsed_time = time.time() - start_time
print(f"rescaling done in {elapsed_time} sec")

print(next(pd.read_csv("numbers_scaled_pandas.csv", header=None, chunksize=5)).iloc[:,:5])
!du numbers_scaled_pandas.csv
!rm numbers_scaled_pandas.csv

Notes:
* You should aim at using as much as memory as possible (but leaving some operational margin).
* Python's `open()` does already buffer I/O reads and writes to minimize number of I/O operations.
* The `pandas` chunks functionality is very convenient, but it does not really offer a speed-up over the plain I/O interface.

### Relation to online algorithms*

**Online algorithm** is one that can process its input piece-by-piece in a serial fashion; the entire input is not available from the start, and algorithm takes action as soon as new data pieces arrive (in contrast to so called *streaming algorithms*).

**Offline algorithm** is given the whole problem data from the start (and is required to output an answer which solves the whole problem).

Online algorithms are considered for real-time/continuously provided data, for instance, for data that are sequentially read from a disk. On the other hand, external memory data processing, in general, considers use of external memory also in parallel, in some structured way, if needed (e.g. queries to databases).


### Memory mapped files

A convenient technique to keep data on disk, but still working with the data **as if it is stored in memory** are [memory mapped files](https://en.wikipedia.org/wiki/Memory-mapped_file).

`numpy` uses [Python `mmap` standard library module](https://docs.python.org/3/library/mmap.html) to support memory mapped files to support work with an array data stored on a disk.

Let's create a large matrix array on a disk in the [NumPy array's binary format `.npy`](https://numpy.org/doc/stable/reference/generated/numpy.lib.format.html#module-numpy.lib.format):

In [ ]:
from numpy.lib.format import open_memmap

out_of_mem_array = open_memmap(
    "large_file.npy",
    mode="w+",
    dtype=np.float64,
    shape=(10_000, 10_000),
)

print(type(out_of_mem_array))

!ls -lh large_file.npy

# alternatively, ask the underlying mmap object for the memory-mapped file size:
print(type(out_of_mem_array._mmap))
print(f"{out_of_mem_array._mmap.size() / (1024 * 1024):.2f}M")

This `10_000 x 10_000` matrix occupies `763M` on hard drive but consumes almost no RAM!

In [ ]:
print(f"     virtual matrix size: {out_of_mem_array.nbytes / (1024 * 1024):.2f}M")
# pympler.asizeof does not work correctly w/ numpy.memmap objects => using __sizeof__ built-in
print(f"numpy.memmap object size: {out_of_mem_array.__sizeof__()} bytes")

Note: measuring size of `numpy.memmap` objects is not useful - there is no direct way to monitor how much of the underlying file is actually mapped into the memory.

We did not initialize any matrix values. By default, it contains only zeros:

In [ ]:
# Beware: just for a demonstration purpose - np.all triggers mapping of
#          the whole array to the memory, which misses the whole point
np.all(out_of_mem_array == 0.0)

We can use this like a regular matrix, but operations will be slowed down due to I/O overhead:

In [ ]:
out_of_mem_array[9, 9] = 1.234

# stay on the safe side: flush changes to disk after done with writing
out_of_mem_array.flush()

Let us open the file in a read-only mode now:

In [ ]:
out_of_mem_array = open_memmap(
    filename="large_file.npy",
    mode="r",
)
print(type(out_of_mem_array))
print(out_of_mem_array.dtype)
print(out_of_mem_array.shape)

Notes:
* we can equivalently use `np.load(..., mmap_mode=...)`;
* using `np.memmap` will save only binary-serialized array, without array metadata header (`shape`, `dtype`) - both constitute the `.npy` binary format.

We can map chunks of array file into RAM and create regular NumPy array out of it, e.g.:

In [ ]:
in_mem_array = np.array(out_of_mem_array[:10, :10])
print(type(in_mem_array))
print(in_mem_array)

*Side note: in general, for an efficient random access to any type of files, instead of reading file chunks into a buffer, you can use the `mmap` standard library module, getting addressing/indexing-based access to file contents (the `mmap` module is also very efficient when reading whole files into the memory; cf. [`mmap` tutorial](https://realpython.com/python-mmap/)).



### Other file formats

- **[HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format)** is a general data format designed to store huge amounts of numerical data as a dictionary of `numpy` arrays.
    * The [`h5py` package](https://docs.h5py.org/en/stable/index.html) can be used to work with HDF5 files in Python; [cf. with the `pytables` package for some more advanced HDF5 features](http://www.pytables.org/FAQ.html#how-does-pytables-compare-with-the-h5py-project).
- **[Apache Parquet](https://en.wikipedia.org/wiki/Apache_Parquet)** is another efficient column/data frame/table-oriented file format
    * Use [Parquet files in Python via the `pyarrow` package](https://arrow.apache.org/docs/python/parquet.html), which integrates with both `numpy` and `pandas`.
- **[SQLite](https://www.sqlite.org/index.html)** offers a SQL database in a single file. SQL database enables efficient storage, extraction and transformation of subsets of data from many, possibly related, large tables/datasets.
    * The [`sqlite3` standard library module](https://docs.python.org/3/library/sqlite3.html) allows to work with SQLite in Python.
    * A [built-in `pandas` support for SQL](https://towardsdatascience.com/python-pandas-and-sqlite-a0e2c052456f) can be used to work with SQL tables as data frames; read *[Fast subsets of large datasets with Pandas and SQLite](https://pythonspeed.com/articles/indexing-pandas-sqlite/)*


## Generators

[Python generator expressions](https://docs.python.org/3/howto/functional.html#generator-expressions-and-list-comprehensions) can produce a stream of data items, fetching items on demand.

Generator expressions look similar to list comprehensions:

In [ ]:
# construct full list of first 1000 square numbers
direct = [i * i for i in range(1000)]

# "lazy list" of first 1000 square numbers
lazy = (i * i for i in range(1000))

Both behave similar in many situations:

In [ ]:
print(sum(direct))
print(sum(lazy))

**BUT**

Generator expressions create generators and generators **don't store objects in memory**. Instead, generators **compute values on demand**:

In [ ]:
from pympler.asizeof import asizeof

direct = [i * i for i in range(1000)]
lazy = (i * i for i in range(1000))

print(f"direct: {asizeof(direct)} bytes, {type(direct)}")
print(f"  lazy: {asizeof(lazy)} bytes, {type(lazy)}")

Like all iterators, generators **can be exhausted**, but you can **iterate through a generator only once**: 

Once all generator items have been computed, iteration will stop and a loop over elements will exit:

In [ ]:
print(sum(lazy))
print(sum(lazy))
print(list(lazy))

Note: use `next()` to get an explicit `StopIteration` exception:

In [ ]:
next(lazy)

In turn, **generators don't support operations such as `len` or indexing/slicing using `[...]`.**

To implement more flexible generators, use functions with the `yield` keyword. This is beyond this course - read [Python documnetation HOWTO on generators](https://docs.python.org/3/howto/functional.html#generators) to learn more about this.

## Further reads

- *[Reducing Pandas memory usage #1: lossless compression](https://pythonspeed.com/articles/pandas-load-less-data/)*
- *[Reducing Pandas memory usage #2: lossy compression](https://pythonspeed.com/articles/pandas-reduce-memory-lossy/)*
- ...
